In [1]:
#nadare_data_dir = "/kaggle/input/nadare-4sq-data"
#nadare_feature_dir = "/kaggle/input/nadare-4sq-data/FEAT20/"
#nadare_util_dir = "/kaggle/input/nadare-4sq-data/common_utils/"
nadare_feature_dir = "../feature/FEAT26/"
nadare_util_dir = "../feature/common_utils/"


In [2]:
!unzip {nadare_util_dir}pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle -d .

unzip:  cannot find or open ../feature/common_utils/pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle, ../feature/common_utils/pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle.zip or ../feature/common_utils/pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle.ZIP.


In [3]:
!conda install ./pykakasi_deps/offline_pykakasi.tar.bz2
!conda install ./pykakasi_deps/offline_jaconv.tar.bz2
!conda install ./pykakasi_deps/offline_deprecated.tar.bz2

/bin/bash: conda: command not found
/bin/bash: conda: command not found
/bin/bash: conda: command not found


In [4]:
!pip install {nadare_util_dir}tensorflow_text-2.6.0-cp37-cp37m-manylinux1_x86_64.whl --no-deps
!pip install {nadare_util_dir}tensorflow_ranking-0.5.0-py2.py3-none-any.whl --no-deps
!pip install {nadare_util_dir}mojimoji-0.0.12-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /feature/common_utils/tensorflow_text-2.6.0-cp37-cp37m-manylinux1_x86_64.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/feature/common_utils/tensorflow_text-2.6.0-cp37-cp37m-manylinux1_x86_64.whl'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /feature/common_utils/tensorflow_ranking-0.5.0-py2.py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/feature/common_utils/tensorflow_ranking-0.5.0-py2.py3-none-any.whl'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /feature/common_utils/mojimoji-0.0.12-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/featur

In [5]:
!pip install {nadare_util_dir}SudachiPy-0.6.5-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl --no-deps
!pip install {nadare_util_dir}SudachiDict_core-20220519-py3-none-any.whl --no-deps
!pip install {nadare_util_dir}SudachiDict_full-20220519-py3-none-any.whl --no-deps
!pip install {nadare_util_dir}pythainlp-3.0.8-py3-none-any.whl --no-deps
!pip install {nadare_util_dir}tinydb-4.7.0-py3-none-any.whl --no-deps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /feature/common_utils/SudachiPy-0.6.5-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/feature/common_utils/SudachiPy-0.6.5-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /feature/common_utils/SudachiDict_core-20220519-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/feature/common_utils/SudachiDict_core-20220519-py3-none-any.whl'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /feature/common_utils/SudachiDict_full-20220519-py3-none-any.whl
ERROR: Could not in

In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text as tf_text

from tqdm.notebook import tqdm
import re

import gc

In [ ]:
import pandas as pd

In [ ]:
#train_df = pd.read_csv("../input/foursquare-location-matching/train.csv", encoding="utf-8")
test_df = pd.read_csv("../input/foursquare-location-matching/train.csv", encoding="utf-8")
#test_df.loc[0, "name"] = np.NaN

FileNotFoundError: ignored

In [ ]:
test_id_map = {v: i for i, v in enumerate(test_df["id"].values)}
test_df["ix"] = test_df["id"].map(test_id_map)
test_df["categories"] = test_df["categories"].fillna("")
test_df["pid"] = -1
test_df["group"] = -1

In [ ]:
from fasttext import load_model
ft_model = load_model(nadare_util_dir + "lid.176.bin")
def predict_language(text):
    label, prob = ft_model.predict(text, 1)
    return list(zip([l.replace("__label__", "") for l in label], prob))[0][0]

test_df["name_language"] = np.vectorize(predict_language)(test_df["name"].fillna(""))
test_df["address_language"] = np.vectorize(predict_language)(test_df["address"].fillna(""))

def language_pattern(country, language):
    if country == "JP":
        return 0
    elif country == "TH":
        return 1
    elif country == "CN":
        return 2
    elif language == "ja":
        return 0
    elif language == "th":
        return 1
    elif language == "zh":
        return 2
    else:
        return 3
test_df["name_handle_pattern"] = np.vectorize(language_pattern)(test_df["country"], test_df["name_language"])     
test_df["address_handle_pattern"] = np.vectorize(language_pattern)(test_df["country"], test_df["address_language"])
lang_vc = test_df["name_language"].value_counts()
del ft_model

In [ ]:
from num2words import num2words
from collections import defaultdict

num2words_languages = ["ar", "cz", "en", "fr", "de", "fi", "es", "id", "kn", "ko", "kz", "lt", "lv", "pl", "ro", "ru", "sl",
                       "sr", "sv", "no", "dk", "pt", "he", "it", "vi", "th", "tr", "nl", "uk", "te", "hu"]

word_number_df = []
for lang in tqdm(num2words_languages):
    for i in range(1001):
        try:
            word = num2words(i, lang=lang)
        except:
            word = np.NaN
        try:
            ordinal = num2words(i, lang=lang, to="ordinal")
        except:
            ordinal = np.NaN
        try:
            ordinal_num = num2words(i, lang=lang, to="ordinal_num")
        except:
            ordinal_num = np.NaN
        if (pd.isna(ordinal)) or (pd.isna(ordinal_num)) or (ordinal == ordinal_num):
            ordinal_num = i
        if lang == "ko" and not pd.isna(ordinal):
            ordinal = ordinal[:-3]
            ordinal_num = ordinal_num[:-3] 
        
        word_number_df.append({"lang": lang,
                               "number": i,
                               "word": word,
                               "ordinal": ordinal,
                               "ordinal_num": ordinal_num})

word_number_df = pd.DataFrame(word_number_df)
word_number_df["lang_count"] = word_number_df["lang"].apply(lambda x: lang_vc.get(x, 0))

word2num_dict = {}
possibly_have_next_dict = {}

for i, row in word_number_df.sort_values(by="lang_count").iterrows():
    if not pd.isna(row["word"]):
        word = row["word"]
        if not word in possibly_have_next_dict.keys():
            possibly_have_next_dict[word] = False

        word2num_dict[word] = str(row["number"])
        word2num_dict[word.replace("-", " ")] = str(row["number"])
        
        wsp = word.split()
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        wsp = word.replace("-", " ").split()[:-1]
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        
    if (not pd.isna(row["ordinal"])) and (not pd.isna(row["ordinal_num"])):
        word = row["ordinal"]
        if not word in possibly_have_next_dict.keys():
            possibly_have_next_dict[word] = False
        word2num_dict[word] = str(row["ordinal_num"])
        word2num_dict[word.replace("-", " ")] = str(row["ordinal_num"])
        
        wsp = word.split()
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        wsp = word.replace("-", " ").split()[:-1]
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        

In [ ]:
import string
import pycnnum
import pythainlp
from pythainlp.util import thai_digit_to_arabic_digit
from sudachipy import Dictionary, SplitMode
import mojimoji
import pykakasi
from unidecode import unidecode
import emoji

emoji_set = set()
for k, v in emoji.UNICODE_EMOJI.items():
    emoji_set.update(v.keys())
thai_word2num_dict = {num2words(i, lang="th"):str(i) for i in range(10001)}
CHINESE_DIGITS_PATTERN = re.compile(r'[〇〇零一二三四五六七八九零壹贰叁肆伍陆柒捌玖十百千万拾佰仟萬]+')
remove_punctuations = '!"#$%\'()*+,./:;<=>?@[\\]^_`{|}~'
punctuation_translater = str.maketrans(remove_punctuations, " "*len(remove_punctuations))
japanese_char2num_translater = str.maketrans("〇零一壱二弐三参四肆五伍六陸七漆八捌九玖", "00112233445566778899")


kakasi = pykakasi.kakasi()
kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
conversion = kakasi.getConverter()

force_katakana = pykakasi.kakasi()
kakasi.setMode('H', 'K')  # Convert Hiragana into alphabet
kakasi.setMode('K', 'K')  # Convert Katakana into alphabet
force_katakana.setMode('J', 'K')  # Convert Hiragana into alphabet
force_katakana_conversion = force_katakana.getConverter()

sudachi_tokenizer = Dictionary(dict_type="full").create()
sudachi_tokenize_mode = SplitMode.A

zh_segmenter = tf_text.HubModuleTokenizer(nadare_util_dir + "zh_segmentation")

def remove_emoji(x):
    return ''.join([c for c in x if c not in emoji_set])

def remove_marks(x):
    x = re.sub(r'[^a-zA-Z0-9& ]', ' ', x)
    x = re.sub(r' +', ' ', x)
    return x

def zenhan_normalize(x):
    return mojimoji.han_to_zen(mojimoji.zen_to_han(x, kana=False), digit=False, ascii=False)

def japanese_wakachi_reading(morphemes):
    readings = []
    for m in morphemes:
        if (m.normalized_form() == " ") or (m.reading_form() == "キゴウ"):
            continue
        if m.normalized_form().endswith("丁目") and m.normalized_form() != "丁目":
            norm = m.normalized_form().translate(japanese_char2num_translater)
            norm = re.sub(r"(\d+)", r" \1 ", "6丁目")
            rs = japanese_wakachi_reading(sudachi_tokenizer.tokenize(norm, sudachi_tokenize_mode))
            readings.extend(rs)
        elif m.normalized_form().isdigit():
            readings.append(m.normalized_form())
        else:
            readings.append(force_katakana_conversion.do(m.reading_form()))
    return readings

def handle_japanese(x):
    x = zenhan_normalize(x)
    morphemes = sudachi_tokenizer.tokenize(x, sudachi_tokenize_mode)
    reading_forms = japanese_wakachi_reading(morphemes)
    romanize_forms = conversion.do(" ".join(reading_forms))
    return " ".join(reading_forms), romanize_forms

def decode_list(x):
    if type(x) is list:
        return list(map(decode_list, x))
    return x.decode("UTF-8")

def decode_utf8_tensor(x):
    return list(map(decode_list, x.to_list()))

def handle_chinese(x):
    x = x.replace("'", "").translate(punctuation_translater).lower()
    if re.fullmatch("\s*", x):
        return "", ""
    with tf.device("CPU:0"):
        words_list = decode_utf8_tensor(zh_segmenter.tokenize(x.split()))
        
    number_words = []
    romanized_words = []
    for words in words_list:
        num_word = []
        roman_word = []
        for word in words:
            if re.fullmatch(CHINESE_DIGITS_PATTERN, word):
                word = str(pycnnum.cn2num(word))
            if (word[0]) == "第" and re.fullmatch(CHINESE_DIGITS_PATTERN, word[1:]):
                num = str(pycnnum.cn2num(word[1:]))
                word = "第"
                num_word.append(word + num)
                roman_word.append(unidecode(word[0]).lower().replace(" ", "") + " " + num)
            else:
                num_word.append(word)
                roman_word.append(unidecode(word).lower().replace(" ", ""))
        number_words.append("".join(num_word))
        romanized_words.append(" ".join(roman_word))
    return " ".join(number_words), " ".join(romanized_words)

def handle_thai(x):
    x = thai_digit_to_arabic_digit(x)
    tokenized_word = pythainlp.tokenize.word_tokenize(x, engine="newmm")
    number_words = []
    romanized_words = []
    for word in tokenized_word:
        word = thai_word2num_dict.get(word, word)
        number_words.append(word)
        try:
            romanized_words.append(pythainlp.romanize(word, engine='royin'))
        except:
            pass
    return "".join(number_words), " ".join(romanized_words)


def language_normalize(word):
    result = []
    word = word.replace("'", "").translate(punctuation_translater).lower()
    tmp = ""
    for w in word.split():
        if len(tmp):
            tmp = tmp + " " + w
            if tmp in possibly_have_next_dict.keys():
                if possibly_have_next_dict[tmp]:
                    continue
                else:
                    result.append(word2num_dict[tmp])
                    tmp = ""
        else:
            if w in possibly_have_next_dict.keys():
                if possibly_have_next_dict[w]:
                    tmp = w
                    continue
                else:
                    result.append(word2num_dict[w])
            else:
                result.append(w)
    if len(tmp):
        result.append(word2num_dict.get(tmp, tmp))
            
    return re.sub(r' +', ' ', " ".join(result))

In [ ]:
def language_handle(handle_pattern, text):
    if handle_pattern == 0:
        normalized, romanized = handle_japanese(text)
    elif handle_pattern == 1:
        normalized, romanized = handle_thai(text)
    elif handle_pattern == 2:
        normalized, romanized = handle_chinese(text)
    return normalized, romanized


In [ ]:
name_handle_index = np.where((test_df["name_handle_pattern"] <= 2) & (~test_df["name"].isna()))[0]

name_normalized = []
name_romanized = []
for pattern, name in tqdm(zip(test_df["name_handle_pattern"].values[name_handle_index],
                              test_df["name"].fillna("").astype(str).values[name_handle_index]), total=len(name_handle_index)):
    normalized, romanized = language_handle(pattern, name)
    name_normalized.append(normalized)
    name_romanized.append(romanized)

In [ ]:
address_handle_index = np.where((test_df["address_handle_pattern"] <= 2) & (~test_df["address"].isna()))[0]

address_normalized = []
address_romanized = []
for pattern, address in tqdm(zip(test_df["address_handle_pattern"].values[address_handle_index],
                              test_df["address"].fillna("").astype(str).values[address_handle_index]), total=len(address_handle_index)):
    normalized, romanized = language_handle(pattern, address)
    address_normalized.append(normalized)
    address_romanized.append(romanized)

In [ ]:
import gc
del zh_segmenter, kakasi, conversion, sudachi_tokenizer
gc.collect()

In [ ]:
test_df["name_normalized"] = np.vectorize(language_normalize)(test_df["name"].fillna("").astype(str))
test_df["address_normalized"] = np.vectorize(language_normalize)(test_df["address"].fillna("").astype(str))


from_cols = ["name_normalized", "address_normalized"]
to_cols = ['sp_name', "sp_address_raw"]
for from_, to_ in zip(from_cols, to_cols):
    test_df[to_] = test_df[from_].fillna('').apply(unidecode)
    test_df[to_] = test_df[to_].str.lower()
    test_df[to_] = test_df[to_].apply(remove_emoji)
    test_df[to_] = test_df[to_].apply(remove_marks)
    
test_df.loc[name_handle_index, "name_normalized"] = name_normalized
test_df.loc[name_handle_index, "sp_name"] = name_romanized

test_df.loc[address_handle_index, "address_normalized"] = address_normalized
test_df.loc[address_handle_index, "sp_address_raw"] = address_romanized

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
address_index = np.where(~np.vectorize(lambda x: bool(re.fullmatch("\s*", x)))(test_df["sp_address_raw"].fillna("")))[0]
knn = KNeighborsRegressor(n_neighbors=min(3, (~test_df["address"].isna()).sum()), 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(test_df[['latitude','longitude']].values)[address_index], address_index)
address_neiighbor = address_index[knn.kneighbors(np.deg2rad(test_df[['latitude','longitude']].values))[1]]
test_df["sp_address"] = np.vectorize(lambda x: " ".join(x))(pd.Series(test_df["sp_address_raw"].values[address_neiighbor].tolist()))


In [ ]:
test_df["sp_name"] = test_df["sp_name"].apply(remove_marks)
test_df["sp_address"] = test_df["sp_address"].apply(remove_marks)

In [ ]:
#test_df[["sp_name", "sp_address", "sp_address_raw"]].to_csv("../processed/sp_input2.csv", index=None)

In [ ]:
#test_df[["sp_name", "sp_address", "sp_address_raw"]] = pd.read_csv("../processed/sp_input2.csv").fillna(" ")

In [ ]:
import os

country_index_df = pd.read_csv(nadare_feature_dir + 'country_tsp_index.csv', encoding="utf-8")
state_index_df = pd.read_csv(nadare_feature_dir + 'state_tsp_index.csv', encoding="utf-8")
city_index_df = pd.read_csv(nadare_feature_dir + 'city_tsp_index.csv', encoding="utf-8")
geo_name_index_df = pd.read_csv(nadare_feature_dir + 'geo_name_tsp_index.csv', encoding="utf-8")
category_index_df = pd.read_csv(nadare_feature_dir + 'category_tsp_index.csv', encoding="utf-8").fillna("nan")


In [ ]:
country_index_map = {country: i for country, i in country_index_df[["country", "country_index"]].values}
state_index_map = {state: i for state, i in state_index_df[["state", "state_index"]].values}
city_index_map = {city: i for city, i in city_index_df[["city", "city_index"]].values}
geo_name_index_map = {geo_name: i for geo_name, i in geo_name_index_df[["geo_name", "geo_name_index"]].values}

In [ ]:
def haversine(X, Y):
    delta = Y - X
    x_lats = tf.gather(X, 0, axis=-1)
    y_lats = tf.gather(Y, 0, axis=-1)
    dlat = tf.gather(delta, 0, axis=-1)
    dlon = tf.gather(delta, 1, axis=-1)

    a = tf.sin(dlat/2) * tf.sin(dlat/2) + tf.cos(x_lats) * tf.cos(y_lats) * tf.sin(dlon/2) * tf.sin(dlon/2)
    c = 2 * tf.math.atan2(tf.sqrt(a), tf.sqrt(1-a))
    return c

In [ ]:
category_ix_dict = {k: v for k, v in category_index_df[["category", "category_ix"]].values}

In [ ]:
len(category_ix_dict)

In [ ]:
def get_value_with_default(dict_, value, default=0):
    return np.vectorize(lambda x: dict_.get(x, default))(value)

def get_category_ix(df, category_ix_dict):
    cat_pairs = []
    all_category = set()
    for row in df["categories"].drop_duplicates().dropna().values:
        for cat in row.split(", "):
            if len(cat):
                cat_pairs.append([row, cat])
                all_category.add(cat)

    category_df = pd.DataFrame(cat_pairs, columns=["categories", "category"])
    categories_df = df[["ix", "categories"]].merge(category_df, on="categories", how="left").sort_values(by="ix")

    categories_df["category_ix"] = get_value_with_default(category_ix_dict, categories_df["category"].values, -1)
    categories_df = categories_df[categories_df["category_ix"] >= 0]
    categories_ix = tf.RaggedTensor.from_value_rowids(values=tf.constant(categories_df["category_ix"].values, "int32"),
                                                              value_rowids=tf.constant(categories_df["ix"].values, "int32"),
                                                              nrows=len(df))
    return categories_ix
category_ix_dict = {k: v for k, v in category_index_df[["category", "category_ix"]].values}
test_categories_ix = get_category_ix(test_df, category_ix_dict)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(city_index_df[['latitude','longitude']]), city_index_df["city_index"])
pseudo_index = knn.kneighbors(np.deg2rad(test_df[['latitude','longitude']]))[1].T[0]
test_df["pseudo_city"] = np.where(test_df["city"].isin(city_index_df["city"].values), test_df["city"].values, city_index_df["city"][pseudo_index].values)
test_df["pseudo_city_ix"] = test_df["pseudo_city"].map(city_index_map)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(city_index_df[['latitude','longitude']]), city_index_df["city_index"])
pseudo_index = knn.kneighbors(np.deg2rad(test_df[['latitude','longitude']]))[1].T[0]
test_df["pseudo_city"] = np.where(test_df["city"].isin(city_index_df["city"].values), test_df["city"].values, city_index_df["city"][pseudo_index].values)
test_df["pseudo_city_ix"] = test_df["pseudo_city"].map(city_index_map)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(state_index_df[['latitude','longitude']]), state_index_df["state_index"])
pseudo_index = knn.kneighbors(np.deg2rad(test_df[['latitude','longitude']]))[1].T[0]
test_df["pseudo_state"] = np.where(test_df["state"].isin(state_index_df["state"].values), test_df["state"].values, state_index_df["state"][pseudo_index].values)
test_df["pseudo_state_ix"] = test_df["pseudo_state"].map(state_index_map)

In [ ]:
geo_name_index_df

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(geo_name_index_df[['latitude','longitude']]), geo_name_index_df["geo_name_index"])
pseudo_index = knn.kneighbors(np.deg2rad(test_df[['latitude','longitude']]))[1].T[0]
test_df["pseudo_geo_name_ix"] = pseudo_index

In [ ]:
test_df["country_ix"] = get_value_with_default(country_index_map, test_df["country"], np.where(country_index_df["country"] == "NAN")[0][0])

In [ ]:
ix_params_dict = {"country": {"dimention": 64, "num_category": len(country_index_df)+1},
                  "state": {"dimention": 64, "num_category": len(state_index_df)},
                  "city": {"dimention": 64, "num_category": len(city_index_df)},
                  "geo_name": {"dimention": 64, "num_category": len(geo_name_index_df)}
                  }
test_ix_values = {"country": tf.convert_to_tensor(test_df["country_ix"].values.astype(np.int32)),
                   "state": tf.convert_to_tensor(test_df["pseudo_state_ix"].values.astype(np.int32)),
                   "city": tf.convert_to_tensor(test_df["pseudo_city_ix"].values.astype(np.int32)),
                   "geo_name": tf.convert_to_tensor(test_df["pseudo_geo_name_ix"].values.astype(np.int32)),
                   }

class IxEmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self, ix_params_dict, num_layer=1, out_dim=128):
        super(IxEmbeddingLayer, self).__init__()
        self.ix_params_dict = ix_params_dict
        self.num_layer = num_layer
        self.out_dim = out_dim
        self.embedding_layers = {k: tf.keras.layers.Embedding(v["num_category"], v["dimention"]) for k, v in ix_params_dict.items()}
        self.denses = [tf.keras.layers.Dense(out_dim, activation="gelu")]
        self.out = tf.keras.layers.Dense(out_dim)
        self.drop_out = tf.keras.layers.Dropout(0.1)
    
    def call(self, ix_values):
        results = []
        for k, v in ix_values.items():
            results.append(self.embedding_layers[k](v))
        X = tf.concat(results, axis=-1)
        
        for i in range(self.num_layer):
            X = self.drop_out(self.denses[i](X))
        return self.out(X)
        

In [ ]:
def approx_ranks(logits):
    r"""Computes approximate ranks given a list of logits.
    Given a list of logits, the rank of an item in the list is one plus the total
    number of items with a larger logit. In other words,
    rank_i = 1 + \sum_{j \neq i} I_{s_j > s_i},
    where "I" is the indicator function. The indicator function can be
    approximated by a generalized sigmoid:
    I_{s_j < s_i} \approx 1/(1 + exp(-(s_j - s_i)/temperature)).
    This function approximates the rank of an item using this sigmoid
    approximation to the indicator function. This technique is at the core
    of "A general approximation framework for direct optimization of
    information retrieval measures" by Qin et al.
    Args:
    logits: A `Tensor` with shape [batch_size, list_size]. Each value is the
      ranking score of the corresponding item.
    Returns:
    A `Tensor` of ranks with the same shape as logits.
    """
    list_size = tf.shape(input=logits)[1]
    x = tf.tile(tf.expand_dims(logits, 2), [1, 1, list_size])
    y = tf.tile(tf.expand_dims(logits, 1), [1, list_size, 1])
    pairs = tf.sigmoid(y - x)
    return tf.reduce_sum(input_tensor=pairs, axis=-1) + .5
    
    
class RankMSE(tf.keras.layers.Layer):
    def __init__(self):
        super(RankMSE, self).__init__()
    
    def call(self, y_true, y_pred):        
        weight = 1 / y_true
        rmse = tf.reduce_sum(tf.math.sqrt(tf.math.square(y_true - y_pred))*weight, axis=-1) / tf.reduce_sum(weight, axis=1)
        
        return tf.reduce_mean(rmse)

In [ ]:
class MixerModel(tf.keras.Model):
    
    def __init__(self, name_model, address_model, ix_emb_layer, cat_emb_layer, num_layer=2, num_dim=128):
        super(MixerModel, self).__init__()
        self.num_layer = num_layer
        self.num_dim = num_dim
        self.name_model = name_model
        self.address_model = address_model
        self.ix_emb_layer = ix_emb_layer
        self.cat_emb_layer = cat_emb_layer
        
        self.tokenize_norm = [tf.keras.layers.LayerNormalization() for i in range(4)]
    
        self.drop_out = tf.keras.layers.Dropout(0.1)
        
        self.filter_denses = [tf.keras.layers.DepthwiseConv2D(kernel_size=(1, 4),
                                                              strides=1,
                                                              padding="valid",
                                                              depth_multiplier=4,
                                                              activation="gelu") for _ in range(self.num_layer)]
        self.out_denses = [tf.keras.layers.Dense(num_dim) for _ in range(self.num_layer)]
        
        #self.pointwise_filter_denses = [tf.keras.layers.Dense(num_dim, activation="gelu") for _ in range(self.num_layer)]
        #self.pointwise_out_denses = [tf.keras.layers.Dense(num_dim) for _ in range(self.num_layer)]
        self.norm_layers = [tf.keras.layers.LayerNormalization() for _ in range(self.num_layer)]
        
    def check_tensor(self, x, ix):
        if isinstance(x, tf.RaggedTensor):
            x = x.to_tensor()
        return x
        
    def call(self, info, training=False):
        ix, name, address, position, ix_values, categories, pids, size = info
        X_name = self.tokenize_norm[0](tf.reduce_sum(self.name_model(name), axis=-2))
        X_address = self.tokenize_norm[1](tf.reduce_sum(self.address_model(address), axis=-2))
        X_ix = self.tokenize_norm[2](self.drop_out(self.ix_emb_layer(ix_values)))
        X_categories = self.tokenize_norm[3](tf.reduce_sum(self.drop_out(self.cat_emb_layer(categories)), axis=-2))
        
        X = tf.stack([X_name, X_address, X_ix, X_categories], axis=-2)
        original_shape = tf.shape(X)
        X = tf.expand_dims(X, axis=1)
        expand_shape = tf.shape(X)
        for i in range(self.num_layer):
            X_ = self.norm_layers[i](X)
            X_ = self.out_denses[i](tf.reshape(self.filter_denses[i](X_), expand_shape))
            X = X + X_
        X = tf.reshape(X, original_shape)
        return tf.reduce_mean(X, axis=-2)

def simcse_task(mixer_model, loss_func, neighbor_info):
    X_a = tf.nn.l2_normalize(mixer_model(neighbor_info, training=True), axis=-1)
    X_b = tf.nn.l2_normalize(mixer_model(neighbor_info, training=True), axis=-1)
    
    X_p = tf.reshape(neighbor_info[-2], [-1, neighbor_info[-1]])
    
    N = tf.shape(X_p)[0]
    R = tf.shape(X_p)[1]
    
    shape = tf.shape(X_a)
    X_a = tf.reshape(X_a, [-1, neighbor_info[-1], shape[-1]])
    X_b = tf.reshape(X_b, [-1, neighbor_info[-1], shape[-1]])
    shape = tf.shape(X_a)
    
    cossim_c = tf.reshape(tf.einsum("nad,nbd->nab", X_a, X_b), [shape[0]*shape[1], shape[1]])
    cossim_r = tf.reshape(tf.einsum("nad,nbd->nab", tf.transpose(X_a, [1, 0, 2]), tf.transpose(X_b, [1, 0, 2])), [shape[0]*shape[1], shape[0]])
 
    mask_c = tf.reshape(tf.expand_dims(X_p, axis=1) != tf.expand_dims(X_p, axis=2), [shape[0]*shape[1], shape[1]])
    mask_r = tf.reshape(tf.expand_dims(X_p, axis=0) != tf.expand_dims(X_p, axis=1), [shape[0]*shape[1], shape[0]])    

    label_c = tf.reshape(tf.repeat(tf.expand_dims(tf.eye(tf.shape(X_a)[1]), axis=0), tf.shape(X_a)[0], axis=0), tf.shape(cossim_c))
    label_r = tf.reshape(tf.repeat(tf.expand_dims(tf.eye(tf.shape(X_a)[0]), axis=1), tf.shape(X_a)[1], axis=0), tf.shape(cossim_r))
    
    mask_c = tf.math.logical_or(label_c == 1., mask_c)
    mask_r = tf.math.logical_or(label_r == 1., mask_r)
    
    cossim_c = tf.where(mask_c, cossim_c, -1.)
    cossim_r = tf.where(mask_r, cossim_r, -1.)
    
    scale_c = tf.sqrt(2.) * tf.math.log(tf.cast(tf.shape(label_c)[1], "float32") - 1.)
    scale_r = tf.sqrt(2.) * tf.math.log(tf.cast(tf.shape(label_r)[1], "float32") - 1.)
    loss = loss_func(label_c, cossim_c * scale_c) + loss_func(label_r, cossim_r * scale_r)
    return loss


In [ ]:
class CategoryEmbeddingLayer(tf.keras.layers.Layer):
    
    def __init__(self, num_categories, dim_categories, init_embedding=None):
        super(CategoryEmbeddingLayer, self).__init__()
        self.num_categories = num_categories
        self.dim_categories = dim_categories
        self.category_dense = tf.Variable(tf.keras.initializers.GlorotUniform()(shape=(num_categories, dim_categories)), trainable=True, name=self.name + "/category_embedding")
        if not init_embedding is None:
            self.category_dense.assign(init_embedding)
            
    def call(self, C):
        X = tf.gather(self.category_dense, C)
        return X

In [ ]:
import tensorflow as tf
from tensorflow_text import SentencepieceTokenizer

class SupervisedContrastiveLoss(tf.keras.layers.Layer):
    def __init__(self, from_logits=False):
        super(SupervisedContrastiveLoss, self).__init__()
        if not from_logits:
            NotImplementedError
        
    @tf.function
    def call(self, true, pred, sample_weight=None):
        if sample_weight is None:
            sample_weight = tf.ones(tf.shape(pred), dtype=tf.float32)
        epred = tf.exp(pred) * sample_weight
        scale = tf.reduce_sum(epred, axis=1, keepdims=True)
        loss = tf.reduce_sum((tf.math.log(scale - epred * true) - true*pred) * true * sample_weight) / tf.reduce_sum(true*sample_weight)
        return loss
    
class SentencePieceEmbeddingLayer(tf.keras.layers.Layer):
    
    def __init__(self, vocab_size, out_dim, sp_model_path, init_embedding=None):
        super(SentencePieceEmbeddingLayer, self).__init__()
        self.vocab_size = vocab_size
        self.out_dim = out_dim
        model = open(sp_model_path, "rb").read()
        self.tokenizer = SentencepieceTokenizer(model)
        self.embedding = tf.keras.layers.Embedding(vocab_size, out_dim)
        if not init_embedding is None:
            self.embedding(0)
            self.embedding.trainable_variables[0].assign(init_embedding)
        self.drop_out = tf.keras.layers.Dropout(0.1)
    
    def call(self, X):
        token = self.tokenizer.tokenize(X)
        X = self.drop_out(self.embedding(token))
        return X
    
class DpConvWrapper(tf.keras.layers.Layer):
    
    def __init__(self, sp_layer, num_layer=2):
        super(DpConvWrapper, self).__init__()
        self.sp_layer = sp_layer
        self.num_layer = num_layer
        self.out_dim = self.sp_layer.out_dim
        self.norms = [tf.keras.layers.LayerNormalization() for i in range(num_layer)]
        self.dep_convs = [tf.keras.layers.DepthwiseConv1D(kernel_size=3, strides=1, padding="same", activation="gelu") for _ in range(num_layer)]
        self.denses = [tf.keras.layers.Dense(sp_layer.out_dim) for _ in range(num_layer)]
        self.drop_out = tf.keras.layers.Dropout(0.1)
        
        
    def call(self, X):
        token = name_dpc.sp_layer.tokenizer.tokenize(X) + 1
        start = tf.cast(tf.concat([tf.zeros([1], dtype="int64"), tf.cumsum(token.row_lengths())], axis=0), "int32")
        row_ids = tf.cast(token.value_rowids(), "int32")
        col_ids = tf.range(tf.shape(token.value_rowids())[0], dtype="int32") - tf.gather(start, token.value_rowids())
        X = self.drop_out(name_dpc.sp_layer.embedding(token.to_tensor(default_value=2)))
        rank = tf.rank(X)
        shape = tf.shape(X)
        
        for i in range(name_dpc.num_layer):
            X_ = name_dpc.norms[i](X)
            X_ = name_dpc.dep_convs[i](X_)
            X_ = name_dpc.denses[i](X_)
            X += X_
            
        X = tf.RaggedTensor.from_value_rowids(tf.gather_nd(X, tf.stack([row_ids, col_ids], axis=1)),
                                              token.value_rowids(),
                                              nrows=tf.cast(shape[0], "int64"))
        return X        

    
class SkipgramModel(tf.keras.Model):
    
    def __init__(self, name_spe):
        super(SkipgramModel, self).__init__()
        self.name_spe = name_spe
        
    #@tf.function(experimental_relax_shapes=True)
    def call(self, name, n_name, n_pid):
        X = skipgram_model.name_spe(name)
        X_sum_neighbor = tf.reduce_sum(skipgram_model.name_spe(tf.slice(n_name, [0, 1], [-1, -1])), axis=2).to_tensor()
        name_skip_loss = skipgram_model.skipgram_task(X, X_sum_neighbor, n_pid)

        return name_skip_loss
    
    #@tf.function(experimental_relax_shapes=True)
    def skipgram_task(self, X, X_sum_neighbor, n_pid):

        my_pid, other_pid = tf.split(n_pid, [1, tf.shape(n_pid)[1]-1], axis=1)
        
        X_sum = tf.reduce_sum(X, axis=1)
        X_sum_other = tf.gather(X_sum, X.value_rowids()) - X.values

        
        X_sum_norm = tf.nn.l2_normalize(X_sum, axis=1)
        X_sum_other_norm = tf.nn.l2_normalize(X_sum_other, axis=1)
        X_sum_neighbor_norm = tf.slice(tf.gather(tf.nn.l2_normalize(X_sum_neighbor, axis=-1), X.value_rowids()), [0, 1, 0], [-1, -1, -1])
        X_value_norm = tf.nn.l2_normalize(X.values, axis=1)
        

        correct_cossim = tf.expand_dims(tf.einsum("Vd,Vd->V", X_value_norm, X_sum_other_norm), axis=1)
        wrong_cossim = tf.einsum("Vd,Nd->VN", X_value_norm, X_sum_norm)
        neighbor_cossim = tf.einsum("Vd,VNd->VN", X_value_norm, X_sum_neighbor_norm)
        cossim = tf.concat([correct_cossim, wrong_cossim, neighbor_cossim], axis=1)

        rowwise_mask = tf.expand_dims(tf.gather(X.row_lengths(), X.value_rowids()) > 1, axis=1)
        sameid_mask = tf.concat([tf.expand_dims(tf.cast(X.value_rowids(), "int32"), axis=1) != tf.expand_dims(tf.range(tf.shape(X.row_lengths())[0] + 1)-1, axis=0),
                                 tf.ones(tf.gather(tf.shape(X_sum_neighbor_norm), [0, 1], axis=0), dtype="bool")], axis=1)
        same_pid_mask = tf.concat([tf.gather(my_pid, X.value_rowids(), axis=0) != tf.linalg.matrix_transpose(my_pid),
                                   tf.gather(my_pid, X.value_rowids()) != tf.gather(other_pid, X.value_rowids())], axis=1)
        
        mask = tf.math.logical_and(tf.math.logical_and(rowwise_mask, sameid_mask), same_pid_mask)
        label = tf.concat([tf.ones(tf.shape(correct_cossim)), tf.zeros(tf.shape(wrong_cossim)), tf.zeros(tf.shape(neighbor_cossim))], axis=1)
        
        # Fixed AdaCos paramet2r
        scale = tf.sqrt(2.) * tf.math.log(tf.cast(tf.shape(label)[1], "float32") - 1.)

        label = tf.where(mask, label, tf.zeros(tf.shape(mask)))
        pred = tf.where(mask, scale * cossim, scale * -1.)
        loss = tf.math.reduce_mean(tf.keras.losses.categorical_crossentropy(label, pred, from_logits=True))
        return loss   
    
class ClassifyTrainModel(tf.keras.Model):
    
    def __init__(self, name_model, address_model, ix_emb_layer, cat_emb_layer):
        super(ClassifyTrainModel, self).__init__()
        self.name_model = name_model
        self.address_model = address_model
        self.ix_emb_layer = ix_emb_layer
        self.cat_emb_layer = cat_emb_layer
        self.dense = tf.keras.layers.Dense(cat_emb_layer.dim_categories, use_bias=True)
        self.scale = tf.sqrt(2.) * tf.math.log(tf.cast(cat_emb_layer.num_categories, "float32") - 1.)
        self.loss_function = SupervisedContrastiveLoss(from_logits=True)
        
        self.layer_norms = [tf.keras.layers.LayerNormalization() for i in range(3)]
        self.drop_out = tf.keras.layers.Dropout(0.1)
        self.nan_mask = tf.concat([tf.zeros([1, 1]), tf.ones([1, cat_emb_layer.num_categories-1])], axis=1)
        
    def transform(self, name, address, ix_values):
        X_name = self.layer_norms[0](tf.reduce_sum(self.drop_out(self.name_model(name)), axis=1))
        X_address = self.layer_norms[1](tf.reduce_sum(self.drop_out(self.address_model(address)), axis=1))
        X_ix = self.layer_norms[2](self.ix_emb_layer(ix_values))
        X = self.dense(tf.concat([X_name, X_address, X_ix], axis=-1))
        return X
        
    def call(self, name, address, ix_values, categories):
        X = self.transform(name, address, ix_values)
        classify_loss = self.classify_task(X, categories)
        return classify_loss
    
    def classify_task(self, X, C):
        cossim = tf.einsum("ND,CD->NC", tf.nn.l2_normalize(X, axis=1), tf.nn.l2_normalize(self.cat_emb_layer.category_dense, axis=1))
        
        true = tf.scatter_nd(tf.stack([C.value_rowids(), C.values], axis=1),
                              tf.ones(tf.shape(C.values)),
                              [tf.shape(X)[0], self.cat_emb_layer.num_categories])
        true = true * self.nan_mask
        calc_loc = tf.where(tf.reduce_sum(true, axis=1) > 0.)
        pred = self.scale * cossim
        loss = self.loss_function(tf.gather_nd(true, calc_loc), tf.gather_nd(pred, calc_loc))
        return loss

In [ ]:
def haversine(X, Y):
    delta = Y - X
    x_lats = tf.gather(X, 0, axis=-1)
    y_lats = tf.gather(Y, 0, axis=-1)
    dlat = tf.gather(delta, 0, axis=-1)
    dlon = tf.gather(delta, 1, axis=-1)

    a = tf.sin(dlat/2) * tf.sin(dlat/2) + tf.cos(x_lats) * tf.cos(y_lats) * tf.sin(dlon/2) * tf.sin(dlon/2)
    c = 2 * tf.math.atan2(tf.sqrt(a), tf.sqrt(1-a))
    return c


class TFMiniBatchHKmeans(tf.keras.Model):
    def __init__(self, n_cluster=64, learning_rate=0.1):
        super(TFMiniBatchHKmeans, self).__init__()
        self.n_cluster = n_cluster
        self.learning_rate = learning_rate
        self.centroids = tf.Variable(tf.zeros((n_cluster, 2), dtype="float32"), trainable=False)
    
    def init_centroid(self, X):
        new_centroids = []
        choosed = tf.zeros(len(X), dtype="bool")

        ix = tf.cast(tf.random.uniform([1], maxval=len(X))//1, "int32")[0]
        choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
        new_centroids.append(tf.gather(X, ix))
        dist = tf.ones([tf.shape(X)[0]]) * tf.float32.max / 10

        for i in tqdm(range(self.n_cluster-1)):
            centroid = tf.gather(X, ix)
    
            dist = tf.where(choosed, tf.keras.backend.epsilon() , tf.minimum(dist, tf.math.square(hkmeans.haversine(centroid, X))))
            prob = dist / tf.reduce_sum(dist, axis=0, keepdims=True)
            logit = tf.math.log(prob) - tf.math.log(1-prob)        
            gumbel = logit - tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(logit))))
            ix = tf.cast(tf.argmax(gumbel), "int32")
            choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
            new_centroids.append(tf.gather(X, ix))
        self.centroids.assign(tf.stack(new_centroids, axis=0))
        
    def partial_fit(self, X):
        score = self.haversine(tf.expand_dims(X, axis=1), tf.expand_dims(self.centroids, axis=0))
        maxix = tf.argmin(score, axis=1)
        new_centroids = tf.math.unsorted_segment_mean(X, maxix, num_segments=self.n_cluster)
        num_member = tf.math.unsorted_segment_sum(tf.ones((len(X), 1)), maxix, num_segments=self.n_cluster)
        new_centroids = tf.where(num_member > 0, new_centroids, self.centroids)
        new_centroids = new_centroids * self.learning_rate + self.centroids * (1 - self.learning_rate)
        dist = tf.reduce_mean(tf.reduce_min(score, axis=1))
        self.centroids.assign(new_centroids)
        return dist
    
    def haversine(self, X, Y):
        delta = Y - X
        x_lats = tf.gather(X, 0, axis=-1)
        y_lats = tf.gather(Y, 0, axis=-1)
        dlat = tf.gather(delta, 0, axis=-1)
        dlon = tf.gather(delta, 1, axis=-1)

        a = tf.sin(dlat/2) * tf.sin(dlat/2) + tf.cos(x_lats) * tf.cos(y_lats) * tf.sin(dlon/2) * tf.sin(dlon/2)
        c = tf.math.atan2(tf.sqrt(a), tf.sqrt(1-a))
        return c
    
    def transform(self, X, return_score=False):
        score = tf.math.log(self.haversine(tf.expand_dims(X, axis=1), tf.expand_dims(self.centroids, axis=0)) + tf.keras.backend.epsilon())
        res = tf.argmin(score, axis=-1)
        if return_score:
            dist = tf.reduce_min(score, axis=-1)
            return res, dist
        else:
            return res

class TFMiniBatchSKmeans(tf.keras.Model):
    def __init__(self, n_cluster=64, n_dim=100, alpha=3/2, learning_rate=0.1):
        super(TFMiniBatchSKmeans, self).__init__()
        self.n_dim = n_dim
        self.n_cluster = n_cluster
        self.alpha = alpha
        self.learning_rate = learning_rate

        self.centroids = tf.Variable(tf.zeros((n_cluster, n_dim), dtype="float32"), trainable=False)
    
    def init_centroid(self, X):
        new_centroids = []
        X = tf.nn.l2_normalize(X, axis=-1)
        choosed = tf.zeros(len(X), dtype="bool")

        ix = tf.cast(tf.random.uniform([1], maxval=len(X))//1, "int32")[0]
        choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
        new_centroids.append(tf.gather(X, ix))
        
        dist = tf.ones([tf.shape(X)[0]]) * tf.float32.max / 10

        for i in tqdm(range(self.n_cluster-1)):
            centroid = tf.gather(X, ix)
            cos = tf.einsum("d,nd->n", centroid, X)
            dist = tf.where(choosed, tf.keras.backend.epsilon() , tf.maximum(tf.keras.backend.epsilon(), tf.minimum(dist, 1. - cos)))         
            prob = dist / tf.reduce_sum(dist, axis=0, keepdims=True)
            logit = tf.math.log(prob) - tf.math.log(1-prob)        
            gumbel = logit - tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(logit))))
            ix = tf.cast(tf.argmax(gumbel), "int32")
            choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
            new_centroids.append(X[ix])
        self.centroids.assign(tf.stack(new_centroids, axis=0))
        
    @tf.function
    def partial_fit(self, X):
        score = tf.einsum("nd,kd->nk", X, self.centroids)
        maxix = tf.argmax(score, axis=1)
        new_centroids = tf.math.unsorted_segment_mean(X, maxix, num_segments=self.n_cluster)
        num_member = tf.math.unsorted_segment_sum(tf.ones((len(X), 1)), maxix, num_segments=self.n_cluster)
        new_centroids = tf.where(num_member > 0, new_centroids, self.centroids)
        new_centroids = tf.nn.l2_normalize(new_centroids, axis=1)
        new_centroids = new_centroids * self.learning_rate + self.centroids * (1 - self.learning_rate)
        new_centroids = tf.nn.l2_normalize(new_centroids, axis=1)
        dist = tf.reduce_mean(self.alpha - tf.reduce_max(score, axis=1))
        self.centroids.assign(new_centroids)
        return dist
    
    def transform(self, X, return_score=False):
        score = tf.einsum("...d,kd->...k", X, self.centroids)
        res = tf.argmax(score, axis=-1)
        if return_score:
            cossim = tf.reduce_max(score, axis=-1)
            return res, cossim
        else:
            return res

In [ ]:
class DataContainer():
    def __init__(self, df, ix_values, category_ix, positive_ix, neighbor_ix, neighbor_dist):
        self.name = tf.constant(df["sp_name"].values)
        self.address = tf.constant(df["sp_address"].values)
        self.position = tf.expand_dims(tf.constant(np.deg2rad(df[["latitude", "longitude"]].astype(np.float32).values), dtype="float32"), axis=0)
        self.pid = tf.constant(df["pid"].astype(np.int32).values, dtype="int32")
        self.group = tf.constant(df["group"].astype(np.int32).values, dtype="int32")
        self.ix_values = ix_values
        self.category_ix = category_ix
        self.positive_ix = positive_ix
        self.neighbor_ix = neighbor_ix
        self.neighbor_dist = neighbor_dist
        
    def get_position(self, ix):
        return tf.gather(tf.gather(self.position, ix, axis=1), 0)

    def call(self, ix, size=1):
        name = tf.gather(self.name, ix)
        address = tf.gather(self.address, ix)
        position = self.get_position(ix)
        categories = tf.gather(self.category_ix, ix)
        ix_values = {k: tf.gather(self.ix_values[k], ix) for k in self.ix_values.keys()}
        pids = tf.gather(self.pid, ix)
        return ix, name, address, position, ix_values, categories, pids, size
    
    @tf.function(experimental_relax_shapes=True)
    def log_haversine(self, X, Y):
        delta = Y - X
        x_lats = tf.gather(X, 0, axis=-1)
        y_lats = tf.gather(Y, 0, axis=-1)
        dlat = tf.gather(delta, 0, axis=-1)
        dlon = tf.gather(delta, 1, axis=-1)

        a = tf.sin(dlat/2) * tf.sin(dlat/2) + tf.cos(x_lats) * tf.cos(y_lats) * tf.sin(dlon/2) * tf.sin(dlon/2)
        c = tf.math.log(2 * tf.math.atan2(tf.sqrt(a), tf.sqrt(1-a)) + tf.keras.backend.epsilon())
        return c
    
    @tf.function(experimental_relax_shapes=True)
    def negative_sample_by_score_top_k(self, ix, score, except_loc=None, k=128, distcut=4):
        pos_ix = tf.gather(self.positive_ix, tf.gather(self.pid, ix))
        score = tf.tensor_scatter_nd_update(score,
                                            tf.stack([tf.cast(pos_ix.value_rowids(), "int32"), pos_ix.values], axis=1),
                                            tf.ones([tf.shape(pos_ix.values)[0]])*tf.float32.min/10.)
        if not except_loc is None:
            score = tf.tensor_scatter_nd_update(score,
                                                except_loc,
                                                tf.ones([tf.shape(except_loc)[0]])*tf.float32.min/10.)            
        score, neighbor = tf.math.top_k(score, k=k)
        _, order = tf.math.top_k(tf.random.uniform([tf.shape(ix)[0], k]), k=k)
        score = tf.gather(score, order, batch_dims=1)
        neighbor = tf.gather(neighbor, order, batch_dims=1)
        
        score = tf.reverse_sequence(score, 
                                    tf.ones([tf.shape(score)[0]], dtype="int32") * tf.shape(score)[1],
                                    seq_axis=1)
        neighbor = tf.reverse_sequence(neighbor,
                                  tf.ones([tf.shape(neighbor)[0]], dtype="int32") * tf.shape(neighbor)[1],
                                  seq_axis=1)
        return [score, neighbor]
    
    def get_positive_info(self, ix):
        pos_ix = tf.gather(self.positive_ix, tf.gather(self.pid, ix))
        Y = self.get_position(pos_ix)
        X = self.get_position(tf.gather(ix, pos_ix.value_rowids()))
        pos_dist = tf.RaggedTensor.from_value_rowids(self.log_haversine(X, Y.values), Y.value_rowids())
        pos_ix = pos_ix.to_tensor(default_value=-1)
        pos_ix = tf.where(tf.expand_dims(ix, axis=1) != pos_ix, pos_ix, -1)
        pos_dist = pos_dist.to_tensor(default_value=0.)
        return pos_dist, pos_ix
    
    @tf.function(experimental_relax_shapes=True)
    def random_walk_sampling(self, query_ix, step, top_k):
        query_ix = tf.cast(query_ix, "int32")
        searched = tf.expand_dims(query_ix, axis=1)
        current = tf.identity(query_ix)

        for i in range(step):
            n_ix = tf.gather(self.neighbor_ix, current)
            n_dist = tf.gather(self.neighbor_dist, current)
            n_dist = tf.where(tf.reduce_any(tf.expand_dims(n_ix, axis=2) == tf.expand_dims(searched, axis=1), axis=2), tf.keras.backend.epsilon(), n_dist)

            prob = n_dist / tf.reduce_sum(n_dist, axis=1, keepdims=True)
            logit = tf.math.log(prob) - tf.math.log(1-prob)        
            gumbel = logit - tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(logit))))
            nexts_ = tf.gather(n_ix, tf.math.top_k(gumbel, k=top_k)[1], batch_dims=1)
            searched = tf.concat([searched, nexts_], axis=1)
            current = tf.gather(nexts_, 0, axis=1)
        return tf.reshape(searched, [-1]), step*top_k + 1

In [ ]:
DIMSIZE = 256
WALK_STEP = 128
WALK_TOPK = 2
num_categories = max(category_ix_dict.values()) + 1

name_spe = SentencePieceEmbeddingLayer(32000, DIMSIZE, nadare_feature_dir + "sp_name.model", None)
address_spe = SentencePieceEmbeddingLayer(32000, DIMSIZE, nadare_feature_dir + "sp_address.model", None)

ix_emb_layer = IxEmbeddingLayer(ix_params_dict, num_layer=1, out_dim=DIMSIZE)
cat_emb_layer = CategoryEmbeddingLayer(num_categories, DIMSIZE)

#locrank_model = LocationModel(address_dpc, ix_emb_layer, num_rank=128)
classify_model = ClassifyTrainModel(name_spe, address_spe, ix_emb_layer, cat_emb_layer)
mixer_model = MixerModel(name_spe, address_spe, ix_emb_layer, cat_emb_layer, num_layer=2, num_dim=DIMSIZE)

mixer_loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=True)


name_spe.embedding(0)
address_spe.embedding(0)
for k, v in ix_emb_layer.embedding_layers.items():
    v(0)

In [ ]:
classify_model.load_weights(nadare_feature_dir + "classify_model")
mixer_model.load_weights(nadare_feature_dir + "mixer_model")

In [ ]:
container_cols = ["sp_name", "sp_address", "latitude", "longitude", "pid", "group"] 
test_container =  DataContainer(test_df[container_cols],
                                   test_ix_values,
                                    test_categories_ix,
                                    None,
                                None,
                                None)
test_ixs = tf.range(len(test_container.name), dtype="int32")

In [ ]:
normalized_category_dense = tf.nn.l2_normalize(cat_emb_layer.category_dense, axis=1)
pseudo_categories = []
category_ix_dict_r = {v: k for k, v in category_ix_dict.items()}
eval_ds = tf.data.Dataset.from_tensor_slices(test_ixs)\
                .batch(128)\
                .map(test_container.call)

for ix, name, address, position, ix_values, categories, pid, size in tqdm(eval_ds):
    X = tf.nn.l2_normalize(classify_model.transform(name, address, ix_values), axis=1)
    label = tf.argmax(tf.einsum("nd,md->nm", X, normalized_category_dense), axis=1)
    for l in label.numpy():
        pseudo_categories.append(category_ix_dict_r[l])
        
def remove_test_only_category(text):
    res = []
    for x in text.split(", "):
        if x in category_ix_dict.keys():
            res.append(x)
    return ", ".join(res)

test_df["categories"] = np.vectorize(remove_test_only_category)(test_df["categories"])
test_df["categories"] = np.where(test_df["categories"] == "", pseudo_categories, test_df["categories"])
test_categories_ix = get_category_ix(test_df, category_ix_dict)

In [ ]:
# name
test_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(test_df)))\
                .batch(1024)\
                .map(test_container.call)

name_embeddings = []
address_embeddings = []
ix_values_embeddings = []
mix_embeddings = []
for query_info in tqdm(test_ds):
    ix, name, address, position, ix_values, categories, pid, size = query_info
    name_embeddings.append(tf.nn.l2_normalize(tf.reduce_sum(name_spe(name), axis=1), axis=1))
    address_embeddings.append(tf.nn.l2_normalize(tf.reduce_sum(address_spe(address), axis=1), axis=1))
    ix_values_embeddings.append(tf.nn.l2_normalize(ix_emb_layer(ix_values), axis=1))
    mix_embeddings.append(tf.nn.l2_normalize(mixer_model(query_info), axis=1))
    

name_embeddings = tf.nn.l2_normalize(tf.concat(name_embeddings, axis=0), axis=1)
address_embeddings = tf.nn.l2_normalize(tf.concat(address_embeddings, axis=0), axis=1)
ix_values_embeddings = tf.nn.l2_normalize(tf.concat(ix_values_embeddings, axis=0), axis=1)
mix_embeddings = tf.nn.l2_normalize(tf.concat(mix_embeddings, axis=0), axis=1)

category_embeddings = tf.nn.l2_normalize(tf.math.reduce_sum(tf.gather(cat_emb_layer.category_dense, test_categories_ix), axis=1), axis=1)


In [ ]:
name_embeddings.shape[0] * name_embeddings.shape[1]

In [ ]:
hkmeans = TFMiniBatchHKmeans(255*8, learning_rate=1.)
name_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(name_embeddings)[1], learning_rate=1.)
address_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(address_embeddings)[1], learning_rate=1.)
category_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(category_embeddings)[1], learning_rate=1.)
ix_values_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(ix_values_embeddings)[1], learning_rate=1.)
mix_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(mix_embeddings)[1], learning_rate=1.)
brand_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(mix_embeddings)[1], learning_rate=1.)
brand_pid_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(mix_embeddings)[1], learning_rate=1.)


In [ ]:
hkmeans.load_weights(nadare_feature_dir + "haversine_kmeans")
name_kmeans.load_weights(nadare_feature_dir + "name_kmeans")
address_kmeans.load_weights(nadare_feature_dir + "address_kmeans")
ix_values_kmeans.load_weights(nadare_feature_dir + "ix_value_kmeans")
mix_kmeans.load_weights(nadare_feature_dir + "mix_kmeans")
category_kmeans.load_weights(nadare_feature_dir + "category_kmeans")
brand_kmeans.load_weights(nadare_feature_dir + "brand_kmeans")
brand_pid_kmeans.load_weights(nadare_feature_dir + "brand_pid_kmeans")


In [ ]:
BATCH_SIZE = 1024
eval_ds = tf.data.Dataset.from_tensor_slices(test_ixs)\
                .batch(10000)
hkmeans_labels = []
name_kmeans_labels = []
address_kmeans_labels = []
ix_values_kmeans_labels = []
mix_kmeans_labels = []
category_kmeans_labels = []
brand_kmeans_labels = []
brand_pid_kmeans_labels = []


hkmeans_scores = []
name_kmeans_scores = []
address_kmeans_scores = []
ix_values_kmeans_scores = []
mix_kmeans_scores = []
category_kmeans_scores = []
brand_kmeans_scores = []
brand_pid_kmeans_scores = []


for ix in tqdm(eval_ds):
    res = hkmeans.transform(tf.gather(test_container.position[0], ix), return_score=True)
    hkmeans_labels.append(res[0])
    hkmeans_scores.append(res[1])
    
    res = name_kmeans.transform(tf.gather(name_embeddings, ix), return_score=True)
    name_kmeans_labels.append(res[0])
    name_kmeans_scores.append(res[1])

    res = address_kmeans.transform(tf.gather(address_embeddings, ix), return_score=True)
    address_kmeans_labels.append(res[0])
    address_kmeans_scores.append(res[1])

    res = ix_values_kmeans.transform(tf.gather(ix_values_embeddings, ix), return_score=True)
    ix_values_kmeans_labels.append(res[0])
    ix_values_kmeans_scores.append(res[1])

    res = mix_kmeans.transform(tf.gather(mix_embeddings, ix), return_score=True)
    mix_kmeans_labels.append(res[0])
    mix_kmeans_scores.append(res[1])
    
    res = category_kmeans.transform(tf.gather(category_embeddings, ix), return_score=True)
    category_kmeans_labels.append(res[0])
    category_kmeans_scores.append(res[1])
    
    res = brand_kmeans.transform(tf.gather(name_embeddings, ix), return_score=True)
    brand_kmeans_labels.append(res[0])
    brand_kmeans_scores.append(res[1]) 
    
    res = brand_pid_kmeans.transform(tf.gather(name_embeddings, ix), return_score=True)
    brand_pid_kmeans_labels.append(res[0])
    brand_pid_kmeans_scores.append(res[1]) 

    
test_df["hkmeans_labels"] = tf.concat(hkmeans_labels, axis=0).numpy().astype(np.int16)
test_df["name_kmeans_labels"] = tf.concat(name_kmeans_labels, axis=0).numpy().astype(np.int16)
test_df["address_kmeans_labels"] = tf.concat(address_kmeans_labels, axis=0).numpy().astype(np.int16)
test_df["ix_values_kmeans_labels"] = tf.concat(ix_values_kmeans_labels, axis=0).numpy().astype(np.int16)
test_df["mix_kmeans_labels"] = tf.concat(mix_kmeans_labels, axis=0).numpy().astype(np.int16)
test_df["category_kmeans_labels"] = tf.concat(category_kmeans_labels, axis=0).numpy().astype(np.int16)
test_df["brand_kmeans_labels"] = tf.concat(brand_kmeans_labels, axis=0).numpy().astype(np.int16)
test_df["brand_pid_kmeans_labels"] = tf.concat(brand_pid_kmeans_labels, axis=0).numpy().astype(np.int16)

test_df["hkmeans_scores"] = tf.concat(hkmeans_scores, axis=0).numpy().astype(np.float32)
test_df["name_kmeans_scores"] = tf.concat(name_kmeans_scores, axis=0).numpy().astype(np.float32)
test_df["address_kmeans_scores"] = tf.concat(address_kmeans_scores, axis=0).numpy().astype(np.float32)
test_df["ix_values_kmeans_scores"] = tf.concat(ix_values_kmeans_scores, axis=0).numpy().astype(np.float32)
test_df["mix_kmeans_scores"] = tf.concat(mix_kmeans_scores, axis=0).numpy().astype(np.float32)
test_df["category_kmeans_scores"] = tf.concat(category_kmeans_scores, axis=0).numpy().astype(np.float32)
test_df["brand_kmeans_scores"] = tf.concat(brand_kmeans_scores, axis=0).numpy().astype(np.float32)
test_df["brand_pid_kmeans_scores"] = tf.concat(brand_pid_kmeans_scores, axis=0).numpy().astype(np.float32)


hkmeans_labels = []
name_kmeans_labels = []
address_kmeans_labels = []
ix_values_kmeans_labels = []
mix_kmeans_labels = []
category_kmeans_labels = []
brand_kmeans_labels = []
brand_pid_kmeans_labels = []


hkmeans_scores = []
name_kmeans_scores = []
address_kmeans_scores = []
ix_values_kmeans_scores = []
mix_kmeans_scores = []
category_kmeans_scores = []
brand_kmeans_scores = []
brand_pid_kmeans_scores = []

import gc
gc.collect()

In [ ]:
import gc
gc.collect()

In [ ]:
del ix_values_kmeans, mix_kmeans_labels
gc.collect()

In [ ]:
# simple version

import tensorflow_ranking as tfr

class LogisticModel(tf.keras.Model):
    def __init__(self, dim):
        super(LogisticModel, self).__init__()
        self.scale = tf.Variable(tf.zeros([dim], dtype="float32"), trainable=True)
        self.loss_func = tfr.keras.losses.ApproxNDCGLoss()
        
    def call(self, X, Y):
        logit = tf.einsum("nmd,d->nm", X, self.scale)# + self.bias
        loss = self.loss_func(Y, logit)
        return loss    
    


In [ ]:
simple_logistic_model = LogisticModel(5)
simple_logistic_model.load_weights(nadare_feature_dir + "simple_logistic_model")


In [ ]:
simple_logistic_model.scale

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
bagofword_vectorizer = TfidfVectorizer(min_df=0, binary=True, use_idf=False, norm="l2", dtype=np.float32)
name_bow = bagofword_vectorizer.fit_transform(test_df["sp_name"]).tocoo()

name_bow_rag_data = tf.RaggedTensor.from_value_rowids(name_bow.data, name_bow.row)
name_bow_rag_col = tf.RaggedTensor.from_value_rowids(name_bow.col, name_bow.row)
name_bow_coo = tf.SparseTensor(tf.cast(tf.stack([name_bow.row, name_bow.col], axis=-1), "int64"), name_bow.data, name_bow.shape)

bigram_word_vectorizer = TfidfVectorizer(min_df=0, binary=True, use_idf=False, norm="l2", dtype=np.float32, ngram_range=(2, 2))
bi_name_bow = bigram_word_vectorizer.fit_transform(test_df["sp_name"]).tocoo()
bi_name_bow_rag_col = tf.RaggedTensor.from_value_rowids(bi_name_bow.col, bi_name_bow.row)

In [ ]:
from sklearn.model_selection import KFold
test_df["pid"] = test_df["point_of_interest"].map({v: i for i, v in enumerate(test_df["point_of_interest"].unique())})

group = -np.ones(len(test_df), dtype="int32")
g = 0
for g, (dev_p_ids, val_p_ids) in enumerate(KFold(n_splits=5, shuffle=True, random_state=42).split(test_df["pid"].unique())):
    ix = test_df["pid"].isin(val_p_ids)
    group[ix] = g
test_df["group"] = group

test_df["pid"] = test_df["point_of_interest"].map({v: i for i, v in enumerate(test_df["point_of_interest"].unique())})

container_cols = ["sp_name", "sp_address", "latitude", "longitude", "pid", "group"] 
test_container =  DataContainer(test_df[container_cols],
                                   test_ix_values,
                                    test_categories_ix,
                                    tf.cast(tf.ragged.constant(test_df.groupby("pid")["ix"].unique().tolist()), "int32"),
                                None,
                                None)

In [ ]:
NUM_CANDIDATE = 32
NEAREST = 4
NAME_SIM = 2
NUM_FEAT = 7
test_ds = tf.data.Dataset.from_tensor_slices(test_ixs)\
                .batch(128)

@tf.function(experimental_relax_shapes=True)
def make_pred_feat(ix):
    selected = tf.expand_dims(ix, axis=1)
    pos_ix = tf.gather(test_container.positive_ix, tf.gather(test_container.pid, ix))

    query_position = tf.expand_dims(test_container.get_position(ix), axis=1)
    dist = test_container.log_haversine(query_position, test_container.position)    
    
    indices = tf.stack([tf.repeat(tf.range(len(ix)), tf.gather(name_bow_rag_data, ix).row_lengths()),
                        tf.gather(name_bow_rag_col, ix).values], axis=-1)
    values = tf.gather(name_bow_rag_data, ix).values
    shape = [len(ix), name_bow_coo.shape[1]]
    query = tf.scatter_nd(indices, values, shape)
    sparse_name_sim = tf.sparse.sparse_dense_matmul(query, name_bow_coo, adjoint_b=True)
        
    name_cossim = tf.einsum("nd,md->nm", tf.gather(name_embeddings, ix), name_embeddings)
    address_cossim = tf.einsum("nd,md->nm", tf.gather(address_embeddings, ix), address_embeddings)
    category_cossim = tf.einsum("nd,md->nm", tf.gather(category_embeddings, ix), category_embeddings)    
    logistic_score = tf.einsum("d,nmd->nm", simple_logistic_model.scale, tf.stack([dist, sparse_name_sim, name_cossim, address_cossim, category_cossim], axis=-1))
    
    
    score, topk = tf.math.top_k(-dist, NEAREST + tf.shape(selected)[1])
    update_pos = tf.where(tf.reduce_any(tf.expand_dims(topk, axis=2) == tf.expand_dims(selected, axis=1), axis=2))
    score = tf.tensor_scatter_nd_update(score, 
                                        update_pos,
                                        tf.ones(tf.shape(update_pos)[0]) * tf.float32.min / 10)
    topk = tf.gather(topk, tf.argsort(-score, axis=1), batch_dims=1)
    selected = tf.concat([selected, tf.slice(topk, [0, 0], [-1, NEAREST])], axis=1)
    
    score, topk = tf.math.top_k(name_cossim, NAME_SIM + tf.shape(selected)[1])
    update_pos = tf.where(tf.reduce_any(tf.expand_dims(topk, axis=2) == tf.expand_dims(selected, axis=1), axis=2))
    score = tf.tensor_scatter_nd_update(score, 
                                        update_pos,
                                        tf.ones(tf.shape(update_pos)[0]) * tf.float32.min / 10)
    topk = tf.gather(topk, tf.argsort(-score, axis=1), batch_dims=1)
    selected = tf.concat([selected, tf.slice(topk, [0, 0], [-1, NAME_SIM])], axis=1)  
    
    score, topk = tf.math.top_k(sparse_name_sim, NAME_SIM + tf.shape(selected)[1])
    update_pos = tf.where(tf.reduce_any(tf.expand_dims(topk, axis=2) == tf.expand_dims(selected, axis=1), axis=2))
    score = tf.tensor_scatter_nd_update(score, 
                                        update_pos,
                                        tf.ones(tf.shape(update_pos)[0]) * tf.float32.min / 10)
    topk = tf.gather(topk, tf.argsort(-score, axis=1), batch_dims=1)
    selected = tf.concat([selected, tf.slice(topk, [0, 0], [-1, NAME_SIM])], axis=1)
    
    score, topk = tf.math.top_k(logistic_score, NUM_CANDIDATE - 2*NAME_SIM - NEAREST + tf.shape(selected)[1])
    update_pos = tf.where(tf.reduce_any(tf.expand_dims(topk, axis=2) == tf.expand_dims(selected, axis=1), axis=2))
    score = tf.tensor_scatter_nd_update(score, 
                                        update_pos,
                                        tf.ones(tf.shape(update_pos)[0]) * tf.float32.min / 10)
    topk = tf.gather(topk, tf.argsort(-score, axis=1), batch_dims=1)
    selected = tf.concat([selected, tf.slice(topk, [0, 0], [-1, NUM_CANDIDATE - 2*NAME_SIM - NEAREST])], axis=1)
   
    
    candidate_ixs  = tf.cast(tf.slice(selected, [0, 1], [-1, -1]), "int32")
    
    candidate_dist = tf.gather(dist, candidate_ixs, batch_dims=1)
    candidate_sparse_name_sim = tf.gather(sparse_name_sim, candidate_ixs, batch_dims=1)
    candidate_name_cossim = tf.gather(name_cossim, candidate_ixs, batch_dims=1)
    candidate_address_cossim = tf.gather(address_cossim, candidate_ixs, batch_dims=1)
    candidate_category_cossim = tf.gather(category_cossim, candidate_ixs, batch_dims=1)
    candidate_logistic_score = tf.gather(logistic_score, candidate_ixs, batch_dims=1)
    candidate_mix_cossim = tf.einsum("nd,nmd->nm", tf.gather(mix_embeddings, ix), tf.gather(mix_embeddings, candidate_ixs))
    
    
    cross_feat = tf.stack([candidate_dist, candidate_sparse_name_sim, candidate_name_cossim, candidate_address_cossim, candidate_category_cossim, candidate_logistic_score, candidate_mix_cossim], axis=-1)
    
    candidate_ixs = tf.reshape(candidate_ixs, [-1])
    cross_feat = tf.reshape(cross_feat, [-1, NUM_FEAT])
    
    return candidate_ixs, cross_feat


query_ixs = []
candidate_ixs = []
cross_feats = []

for query_ix in tqdm(test_ds):
    query_ix = tf.reshape(query_ix, [-1])
    candidate_ix, cross_feat = make_pred_feat(query_ix)
    
    query_ixs.append(query_ix)
    candidate_ixs.append(candidate_ix)
    cross_feats.append(cross_feat)
    

In [ ]:
query_ixs = tf.repeat(tf.concat(test_ixs, axis=0), NUM_CANDIDATE, axis=0)
candidate_ixs = tf.concat(candidate_ixs, axis=0)
cross_feats = tf.concat(cross_feats, axis=0)


In [ ]:
import tensorflow_text as tf_text
wsp_tokenizer = tf_text.WhitespaceTokenizer()
uch_tokenizer = tf_text.UnicodeCharTokenizer()
rouge_l = tf_text.metrics.rouge_l

In [ ]:
import phonenumbers
from urllib.parse import urlparse

def get_phene_info(phone, country):
    if phone == "":
        return {"phone_country_code": 0, "national_number": 0}
    if phonenumbers.country_code_for_region(country) == 0:
        country = None
    try:
        res = phonenumbers.parse(phone, country)
    except:
        return {"phone_country_code": 0, "national_number": 0}
    return {"phone_country_code": res.country_code, "national_number": res.national_number}
res = []
for phone, country in test_df[["phone", "country"]].values:
    res.append(get_phene_info(phone, country))
phone_df = pd.DataFrame(res)

test_df["url"] = test_df["url"].fillna("")
test_df["phone_national_number"] = phone_df["national_number"].astype(str)
test_df["url_netloc"] = test_df["url"].fillna("").apply(lambda x: urlparse(x)[1])
test_df["url_path"] = test_df["url"].fillna("").apply(lambda x: urlparse(x)[2])


In [ ]:
def change_number(text):
    res = []
    for word in text.split():
        if word.isnumeric():
            res.append(word)
        else:
            for r in re.findall(r"\d+", str(word)):
                res.append(r)
    return " ".join(res)

test_df["name_numbers"] = np.vectorize(change_number)(test_df["name_normalized"].fillna("").astype(str))
test_df["address_numbers"] = np.vectorize(change_number)(test_df["address_normalized"].fillna("").astype(str))

In [ ]:
from polyleven import levenshtein
from cdifflib import CSequenceMatcher
from jarowinkler import jarowinkler_similarity, jaro_similarity

In [ ]:
from joblib import Parallel, delayed
import Levenshtein
from difflib import SequenceMatcher
from polyleven import levenshtein
from cdifflib import CSequenceMatcher
from jarowinkler import jarowinkler_similarity, jaro_similarity


def make_data(query_ix, candidate_ix):
    query_info = {}
    candidate_info = {}
    query_info["ix"] = query_ix
    candidate_info["ix"] = candidate_ix
    
    for col in ["sp_name", "name_normalized", "sp_address_raw", "phone_national_number", "url", "phone_national_number",
                "url_netloc", "name_numbers", "address_numbers"]:
        query_info[col] = test_df.at[query_ix, col]
        candidate_info[col] = test_df.at[candidate_ix, col]
    query_info["name_numbers_split_size"] = len(query_info["name_numbers"].split())
    candidate_info["name_numbers_split_size"] = len(candidate_info["name_numbers"].split())
    query_info["address_numbers_split_size"] = len(query_info["address_numbers"].split())
    candidate_info["address_numbers_split_size"] = len(candidate_info["address_numbers"].split())    
        
    return query_info, candidate_info

def add_features(query_info, candidate_info):
    res = []
    res.append(query_info["ix"])
    res.append(candidate_info["ix"])
    
    for col in ["sp_name", "name_normalized", "sp_address_raw"]:
        s = query_info[col]
        match_s = candidate_info[col]
        if len(s) and len(match_s):
            res.append(abs(len(match_s) - len(s)))
            res.append(abs(len(match_s.split()) - len(s.split())))
            res.append(CSequenceMatcher(None, s, match_s).ratio())
            res.append(levenshtein(s, match_s))
            res.append(res[-1] / (max(len(s), len(match_s)) + 1e-9))
            res.append(Levenshtein.jaro_winkler(s, match_s))
#             res.append(jarowinkler_similarity(s, match_s))
#             res.append(jaro_similarity(s, match_s))
        else:
            res.append(abs(len(match_s) - len(s)))
            res.append(abs(len(match_s.split()) - len(s.split())))
            res.append(np.nan)
            res.append(np.nan)
            res.append(res[-1] / (max(len(s), len(match_s)) + 1e-9))
            res.append(np.nan)
    
    for col in ["phone_national_number", "url", "url_netloc"]:
        r = 0
        if min(len(query_info[col]), len(candidate_info[col])) > 0:
            r = -1 + 2 * (query_info[col] == candidate_info[col])
        res.append(r)
    for col in ["name_numbers", "address_numbers"]:
        res.append(max(query_info[col + "_split_size"], candidate_info[col + "_split_size"]))
        res.append(min(query_info[col + "_split_size"], candidate_info[col + "_split_size"]))
        s = query_info[col]
        match_s = candidate_info[col]
        if s != '' and match_s != '':
            res.append(SequenceMatcher(None, query_info[col], candidate_info[col]).ratio())
        else:
            res.append(np.nan)
    
    return res

In [ ]:
print(len(query_ixs))
results = Parallel(n_jobs=-1, verbose=1)(delayed(add_features)(*make_data(q_ix, c_ix)) for q_ix, c_ix in zip(query_ixs.numpy(), candidate_ixs.numpy()))
candidate_data_df = pd.DataFrame(results, columns = ["query_ix",
                                                     "candidate_ix",
                                                     "name_delta_len",
                                                     "name_delta_words",
                                                     "name_gesh", 
                                                     "name_leven", 
                                                     "name_nleven", 
                                                     "name_jaro",                                                     
                                                     "name_normalized_delta_len",
                                                     "name_normalized_delta_words",
                                                     "name_normalized_gesh", 
                                                     "name_normalized_leven", 
                                                     "name_normalized_nleven", 
                                                     "name_normalized_jaro", 
                                                     "address_delta_len", 
                                                     "address_delta_words", 
                                                     "address_gesh", 
                                                     "address_leven", 
                                                     "address_nleven", 
                                                     "address_jaro",
                                                     "phone_national_number_match",
                                                     "url_match",
                                                     "url_netloc_match",
                                                     "name_numbers_max",
                                                     "name_numbers_min",
                                                     "name_numbers_gesh",
                                                     "address_numbers_max",
                                                     "address_numbers_min",
                                                     "address_numbers_gesh",])

In [ ]:
#del results
#gc.collect()

int64_cols = ['name_delta_len', 'name_delta_words', 
              'name_normalized_delta_len', 'name_normalized_delta_words',
              'address_delta_len','address_delta_words', 
              'phone_national_number_match', 'url_match', 'url_netloc_match',
              'name_numbers_max', 'name_numbers_min',
              'address_numbers_max', 'address_numbers_min']
float64_cols = ['name_gesh', 'name_nleven', 'name_jaro', 'name_leven',
                'name_normalized_gesh', 'name_normalized_nleven', 'name_normalized_jaro', 'name_normalized_leven',
                'address_gesh','address_nleven', 'address_jaro', 'address_leven',
                'name_numbers_gesh','address_numbers_gesh']
candidate_data_df[int64_cols] = candidate_data_df[int64_cols].astype(np.int16)
candidate_data_df[float64_cols] = candidate_data_df[float64_cols].astype(np.float32)


In [ ]:
gc.collect()

In [ ]:
sp_names = tf.constant(test_df["sp_name"].values)
normalized_names = tf.constant(test_df["name_normalized"].values)
sp_address = tf.constant(test_df["sp_address_raw"].values)
normalized_address = tf.constant(test_df["address_normalized"].values)


In [ ]:
@tf.function(experimental_relax_shapes=True)
def calc_rouge_1_2(q_ix, c_ix):
    q_code = tf.gather(name_bow_rag_col, q_ix).to_tensor(default_value=-1)
    c_code = tf.gather(name_bow_rag_col, c_ix).to_tensor(default_value=-2)
    q_cnt = tf.cast(tf.gather(name_bow_rag_col.row_lengths(), q_ix), "float32")
    c_cnt = tf.cast(tf.gather(name_bow_rag_col.row_lengths(), c_ix), "float32")

    is_na = tf.minimum(q_cnt, c_cnt) == 0.
    match = tf.expand_dims(q_code, axis=2) == tf.expand_dims(c_code, axis=1)
    precision = tf.math.count_nonzero(tf.math.reduce_any(match, axis=2), axis=1, dtype="float32") / (q_cnt + tf.keras.backend.epsilon())
    recall = tf.math.count_nonzero(tf.math.reduce_any(match, axis=1), axis=1, dtype="float32") / (c_cnt + tf.keras.backend.epsilon())
    harmony = 2. / ((1 / (precision + tf.keras.backend.epsilon())) + (1 / (recall + tf.keras.backend.epsilon())))
    res_1 = tf.where(tf.expand_dims(is_na, axis=-1), np.nan, tf.stack([harmony, precision, recall], axis=1))
    
    q_code = tf.gather(bi_name_bow_rag_col, q_ix).to_tensor(default_value=-1)
    c_code = tf.gather(bi_name_bow_rag_col, c_ix).to_tensor(default_value=-2)
    q_cnt = tf.cast(tf.gather(bi_name_bow_rag_col.row_lengths(), q_ix), "float32")
    c_cnt = tf.cast(tf.gather(bi_name_bow_rag_col.row_lengths(), c_ix), "float32")

    is_na = tf.minimum(q_cnt, c_cnt) == 0.
    match = tf.expand_dims(q_code, axis=2) == tf.expand_dims(c_code, axis=1)
    precision = tf.math.count_nonzero(tf.math.reduce_any(match, axis=2), axis=1, dtype="float32") / (q_cnt + tf.keras.backend.epsilon())
    recall = tf.math.count_nonzero(tf.math.reduce_any(match, axis=1), axis=1, dtype="float32") / (c_cnt + tf.keras.backend.epsilon())
    harmony = 2. / ((1 / (precision + tf.keras.backend.epsilon())) + (1 / (recall + tf.keras.backend.epsilon())))
    res_2 = tf.where(tf.expand_dims(is_na, axis=-1), np.nan, tf.stack([harmony, precision, recall], axis=1))
    
    return tf.concat([res_1, res_2], axis=-1)

@tf.function(experimental_relax_shapes=True)
def calc_rouge_l(q_ix, c_ix):
    q_name = wsp_tokenizer.tokenize(tf.gather(sp_names, q_ix))
    c_name = wsp_tokenizer.tokenize(tf.gather(sp_names, c_ix))
    name_score = tf.stack(tf_text.metrics.rouge_l(q_name, c_name), axis=-1)
    q_name = uch_tokenizer.tokenize(tf.gather(normalized_names, q_ix))
    c_name = uch_tokenizer.tokenize(tf.gather(normalized_names, c_ix))
    n_name_score = tf.stack(tf_text.metrics.rouge_l(q_name, c_name), axis=-1)
    
    q_address = wsp_tokenizer.tokenize(tf.gather(sp_address, q_ix))
    c_address = wsp_tokenizer.tokenize(tf.gather(sp_address, c_ix))
    address_score = tf.stack(tf_text.metrics.rouge_l(q_address, c_address), axis=-1)
    q_address = uch_tokenizer.tokenize(tf.gather(normalized_address, q_ix))
    c_address = uch_tokenizer.tokenize(tf.gather(normalized_address, c_ix))
    n_address_score = tf.stack(tf_text.metrics.rouge_l(q_address, c_address), axis=-1)
    score = tf.concat([name_score, n_name_score, address_score, n_address_score], axis=-1)
    return score

In [ ]:
rouge_l_

In [ ]:
text_ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(query_ixs), tf.data.Dataset.from_tensor_slices(candidate_ixs)))\
                         .batch(1024)
wsp_tokenizer = tf_text.WhitespaceTokenizer()
uch_tokenizer = tf_text.UnicodeCharTokenizer()

n_scores = []
l_scores = []
for q_ix, c_ix in tqdm(text_ds):
    n_scores.append(calc_rouge_1_2(q_ix, c_ix))
    l_scores.append(calc_rouge_l(q_ix, c_ix))

n_scores = tf.concat(n_scores, axis=0)
l_scores = tf.concat(l_scores, axis=0)
candidate_data_df[["rouge_1_name_f", "rouge_1_name_p", "rouge_1_name_r",
                   "rouge_2_name_f", "rouge_2_name_p", "rouge_2_name_r"]] = n_scores.numpy().astype(np.float32)
candidate_data_df[["name_f", "name_p", "name_r","n_name_f", "n_name_p", "n_name_r",
                   "address_f", "address_p", "address_r","n_address_f", "n_address_p", "n_address_r"]] = l_scores.numpy().astype(np.float32)

In [ ]:
old_names = ["name_f", "name_p", "name_r","n_name_f", "n_name_p", "n_name_r",
                   "address_f", "address_p", "address_r","n_address_f", "n_address_p", "n_address_r"]
new_names = ["rouge_l_name_f", "rouge_l_name_p", "rouge_l_name_r","rouge_l_n_name_f", "rouge_l_n_name_p", "rouge_l_n_name_r",
                   "rouge_l_address_f", "rouge_l_address_p", "rouge_l_address_r","rouge_l_n_address_f", "rouge_l_n_address_p", "rouge_l_n_address_r"]

candidate_data_df = candidate_data_df.rename({o:n for o, n in zip(old_names, new_names)}, axis=1)

In [ ]:
candidate_data_df.columns.tolist()

In [ ]:
del scores
gc.collect()

In [ ]:
candidate_data_df["distance"] = tf.reshape(tf.gather(cross_feats, 0, axis=1), [-1]).numpy().astype(np.float32)
candidate_data_df["sparse_name_sim"] = tf.reshape(tf.gather(cross_feats, 1, axis=1), [-1]).numpy().astype(np.float32)
candidate_data_df["name_cossim"] = tf.reshape(tf.gather(cross_feats, 2, axis=1), [-1]).numpy().astype(np.float32)
candidate_data_df["address_cossim"] = tf.reshape(tf.gather(cross_feats, 3, axis=1), [-1]).numpy().astype(np.float32)
candidate_data_df["category_cossim"] = tf.reshape(tf.gather(cross_feats, 4, axis=1), [-1]).numpy().astype(np.float32)
candidate_data_df["mix_cossim"] = tf.reshape(tf.gather(cross_feats, 5, axis=1), [-1]).numpy().astype(np.float32)
candidate_data_df["logistic_score"] = tf.reshape(tf.gather(cross_feats, 6, axis=1), [-1]).numpy().astype(np.float32)

In [ ]:
pd.to_pickle(candidate_data_df, "candidate_df"

In [ ]:
pd.to_pickle(candidate_data_df, "../processed/candidate_df_pred23.pickle", protocol=4)

In [ ]:
city_index_df

In [ ]:
category_tsp_cols = ["city_pos_tsp_index", "state_pos_tsp_index", "geo_name_pos_tsp_index"]
kmeans_labels_cols = ['hkmeans_labels', 'name_kmeans_labels', 'address_kmeans_labels', 'ix_values_kmeans_labels', "mix_kmeans_labels", "category_kmeans_labels", 'brand_kmeans_labels', "brand_pid_kmeans_labels"]
kmeans_scores_cols = ['hkmeans_scores', 'name_kmeans_scores', 'address_kmeans_scores', 'ix_values_kmeans_scores', "mix_kmeans_scores", "category_kmeans_scores", 'brand_kmeans_scores', "brand_pid_kmeans_scores"]

candidate_data_df["query_city_index"] = test_df["pseudo_city_ix"].values.astype(np.int16)[candidate_data_df["query_ix"]]
candidate_data_df["query_geo_name_index"] = test_df["pseudo_geo_name_ix"].values.astype(np.int16)[candidate_data_df["query_ix"]]
candidate_data_df["query_country_index"] = test_df["country_ix"].values.astype(np.int16)[candidate_data_df["query_ix"]]
candidate_data_df["query_state_index"] = test_df["pseudo_state_ix"].values.astype(np.int16)[candidate_data_df["query_ix"]]

#candidate_data_df["query_city_emb_tsp_index"] = city_index_df["city_emb_tsp_index"].values.astype(np.int16)[candidate_data_df["query_city_index"]]
#candidate_data_df["query_state_emb_tsp_index"] = state_index_df["state_emb_tsp_index"].values.astype(np.int16)[candidate_data_df["query_state_index"]]
candidate_data_df["query_city_pos_tsp_index"] = city_index_df["city_pos_tsp_index"].values.astype(np.int16)[candidate_data_df["query_city_index"]]
candidate_data_df["query_geo_name_pos_tsp_index"] = geo_name_index_df["geo_name_pos_tsp_index"].values.astype(np.int16)[candidate_data_df["query_geo_name_index"]]
candidate_data_df["query_state_pos_tsp_index"] = state_index_df["state_pos_tsp_index"].values.astype(np.int16)[candidate_data_df["query_state_index"]]

for col in kmeans_labels_cols:
    candidate_data_df[f"query_{col}"] = test_df[col].values.astype(np.int16)[candidate_data_df["query_ix"]]
for col in kmeans_scores_cols:
    candidate_data_df[f"query_{col}"] = test_df[col].values.astype(np.float32)[candidate_data_df["query_ix"]]

candidate_data_df["candidate_city_index"] = test_df["pseudo_city_ix"].values.astype(np.int16)[candidate_data_df["candidate_ix"]]
candidate_data_df["candidate_geo_name_index"] = test_df["pseudo_geo_name_ix"].values.astype(np.int16)[candidate_data_df["candidate_ix"]]

candidate_data_df["candidate_country_index"] = test_df["country_ix"].values.astype(np.int16)[candidate_data_df["candidate_ix"]]
candidate_data_df["candidate_state_index"] = test_df["pseudo_state_ix"].values.astype(np.int16)[candidate_data_df["candidate_ix"]]

#andidate_data_df["candidate_city_emb_tsp_index"] = city_index_df["city_emb_tsp_index"].values.astype(np.int16)[candidate_data_df["candidate_city_index"]]
#candidate_data_df["candidate_state_emb_tsp_index"] = state_index_df["state_emb_tsp_index"].values.astype(np.int16)[candidate_data_df["candidate_state_index"]]
candidate_data_df["candidate_city_pos_tsp_index"] = city_index_df["city_pos_tsp_index"].values.astype(np.int16)[candidate_data_df["candidate_city_index"]]
candidate_data_df["candidate_geo_name_pos_tsp_index"] = geo_name_index_df["geo_name_pos_tsp_index"].values.astype(np.int16)[candidate_data_df["candidate_geo_name_index"]]
candidate_data_df["candidate_state_pos_tsp_index"] = state_index_df["state_pos_tsp_index"].values.astype(np.int16)[candidate_data_df["candidate_state_index"]]

for col in kmeans_labels_cols:
    candidate_data_df[f"candidate_{col}"] = test_df[col].values.astype(np.int16)[candidate_data_df["candidate_ix"]]
for col in kmeans_scores_cols:
    candidate_data_df[f"candidate_{col}"] = test_df[col].values.astype(np.float32)[candidate_data_df["candidate_ix"]]


for col in kmeans_labels_cols + category_tsp_cols:
    candidate_data_df[f"{col}_1d_dist"] = np.abs(candidate_data_df[f"query_{col}"] - candidate_data_df[f"candidate_{col}"])

In [ ]:
feature_cols = [ 'name_delta_len',
 'name_delta_words',
 'name_gesh',
 'name_leven',
 'name_nleven',
 'name_jaro',
 'name_normalized_delta_len',
 'name_normalized_delta_words',
 'name_normalized_gesh',
 'name_normalized_leven',
 'name_normalized_nleven',
 'name_normalized_jaro',
 'address_delta_len',
 'address_delta_words',
 'address_gesh',
 'address_leven',
 'address_nleven',
 'address_jaro',
 'phone_national_number_match',
 'url_match',
 'url_netloc_match',
 'name_numbers_max',
 'name_numbers_min',
 'name_numbers_gesh',
 'address_numbers_max',
 'address_numbers_min',
 'address_numbers_gesh',
 'rouge_1_name_f',
 'rouge_1_name_p',
 'rouge_1_name_r',
 'rouge_2_name_f',
 'rouge_2_name_p',
 'rouge_2_name_r',
 'rouge_l_name_f',
 'rouge_l_name_p',
 'rouge_l_name_r',
 'rouge_l_n_name_f',
 'rouge_l_n_name_p',
 'rouge_l_n_name_r',
 'rouge_l_address_f',
 'rouge_l_address_p',
 'rouge_l_address_r',
 'rouge_l_n_address_f',
 'rouge_l_n_address_p',
 'rouge_l_n_address_r',
 'distance',
 'sparse_name_sim',
 'name_cossim',
 'address_cossim',
 'category_cossim',
 'mix_cossim',
 'logistic_score',
 'query_city_index',
 'query_geo_name_index',
 'query_country_index',
 'query_state_index',
 'query_city_pos_tsp_index',
 'query_geo_name_pos_tsp_index',
 'query_state_pos_tsp_index',
 'query_hkmeans_labels',
 'query_name_kmeans_labels',
 'query_address_kmeans_labels',
 'query_ix_values_kmeans_labels',
 'query_mix_kmeans_labels',
 'query_category_kmeans_labels',
 'query_brand_kmeans_labels',
 'query_brand_pid_kmeans_labels',
 'query_hkmeans_scores',
 'query_name_kmeans_scores',
 'query_address_kmeans_scores',
 'query_ix_values_kmeans_scores',
 'query_mix_kmeans_scores',
 'query_category_kmeans_scores',
 'query_brand_kmeans_scores',
 'query_brand_pid_kmeans_scores',
 'candidate_city_index',
 'candidate_geo_name_index',
 'candidate_country_index',
 'candidate_state_index',
 'candidate_city_pos_tsp_index',
 'candidate_geo_name_pos_tsp_index',
 'candidate_state_pos_tsp_index',
 'candidate_hkmeans_labels',
 'candidate_name_kmeans_labels',
 'candidate_address_kmeans_labels',
 'candidate_ix_values_kmeans_labels',
 'candidate_mix_kmeans_labels',
 'candidate_category_kmeans_labels',
 'candidate_brand_kmeans_labels',
 'candidate_brand_pid_kmeans_labels',
 'candidate_hkmeans_scores',
 'candidate_name_kmeans_scores',
 'candidate_address_kmeans_scores',
 'candidate_ix_values_kmeans_scores',
 'candidate_mix_kmeans_scores',
 'candidate_category_kmeans_scores',
 'candidate_brand_kmeans_scores',
 'candidate_brand_pid_kmeans_scores',
 'hkmeans_labels_1d_dist',
 'name_kmeans_labels_1d_dist',
 'address_kmeans_labels_1d_dist',
 'ix_values_kmeans_labels_1d_dist',
 'mix_kmeans_labels_1d_dist',
 'category_kmeans_labels_1d_dist',
 'brand_kmeans_labels_1d_dist',
 'brand_pid_kmeans_labels_1d_dist',
 'city_pos_tsp_index_1d_dist',
 'state_pos_tsp_index_1d_dist',
 'geo_name_pos_tsp_index_1d_dist']

In [ ]:
test_df["true_count"] = test_df.groupby("pid")["ix"].transform("count")
candidate_data_df["label"] = test_df["pid"][candidate_data_df["query_ix"].values].values == test_df["pid"][candidate_data_df["candidate_ix"].values].values
candidate_data_df["detected"] = True
candidate_data_df["index"] = candidate_data_df.index
candidate_data_df["rank"] = candidate_data_df.groupby("query_ix")["index"].rank("first")

In [ ]:
dev_ixs = np.where(test_df["group"] != 0)[0]
val_ixs = np.where(test_df["group"] == 0)[0]

In [ ]:
from itertools import groupby

class UnionFind():
    
    def __init__(self, N):
        self.parent = [-1] * N
        self.size = [1] * N
        
    def find(self, x):
        p = self.parent[x]
        if p == -1:
            return x
        p = self.find(p)
        self.parent[x] = p
        return p
    
    def unite(self, x, y):
        px = self.find(x)
        py = self.find(y)
        if px == py:
            return
        if self.size[px] < self.size[py]:
            px, py = py, px
        self.size[px] += self.size[py]
        self.parent[py] = px



In [ ]:
dev_loc = (candidate_data_df["query_ix"].isin(dev_ixs) & (~candidate_data_df["candidate_ix"].isin(val_ixs)) &
           (candidate_data_df["detected"]) & (candidate_data_df["rank"] <= 32))
val_loc = (candidate_data_df["query_ix"].isin(val_ixs) & (candidate_data_df["detected"])  &
           (candidate_data_df["rank"] <= 32))


dev_data_df = candidate_data_df[dev_loc]
val_data_df = candidate_data_df[val_loc]
dev_data_df["true_count"] = test_df["true_count"].values[dev_data_df["query_ix"].values]

dev_data_df["weight"] = np.where(dev_data_df["label"],
                                 1 - (dev_data_df["true_count"] - 1) / dev_data_df["true_count"],
                                 1 - (dev_data_df["true_count"]) / (dev_data_df["true_count"]+1))
dev_data_df["weight"] = dev_data_df["weight"] / dev_data_df["weight"].mean()

In [ ]:
val_data_df["rank"].max()

In [ ]:
import lightgbm as lgb

lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 2**11 - 1,
    "learning_rate" : 0.1,
    "bagging_freq": 1,
     "bin_construct_sample_cnt": 200000,
    "lambda_l1": 1.,
    "lambda_l2": 1.,
    "bagging_fraction" : 0.9,
    "feature_fraction" : 0.8,
    "seed": 0}

dev_data = lgb.Dataset(dev_data_df[feature_cols], label=dev_data_df["label"].astype(np.float32), weight=dev_data_df["weight"].values)
val_data = lgb.Dataset(val_data_df[feature_cols], label=val_data_df["label"].astype(np.float32))

lgb_model = lgb.train(lgb_params,
                      dev_data,
                      100000,
                      valid_sets = [dev_data, val_data],
                      early_stopping_rounds=10,
                      verbose_eval = 10
                      )
f_df = pd.DataFrame(lgb_model.feature_importance(importance_type="gain"), index=feature_cols).sort_values(by=0, ascending=False)


val_data_df["pred"] = lgb_model.predict(val_data_df[feature_cols])

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import warnings
from heapq import heappop, heappush
from collections import defaultdict
from tqdm.notebook import tqdm
warnings.simplefilter('ignore')

max_dist = 3
max_N = 64
eval_points = val_data_df["query_ix"].unique()
pids = test_df["pid"].values

best_score = 0
best_th = 0

pred_df = val_data_df[["query_ix", "candidate_ix", "pred", "rank"]]#.query("rank <= 16")

for th in tqdm(np.linspace(0.5, 0.9, 40)):
    tmp_df = pred_df[pred_df["pred"] >= th][["query_ix", "candidate_ix", "pred"]]
    uft = UnionFind(len(test_df))

    for ix, nix in tmp_df[["query_ix", "candidate_ix"]].values:
        uft.unite(ix, nix)
    group_members = defaultdict(list)
    group_size = defaultdict(int)
    group_map = {}
    for i in range(len(test_df)):
        group_members[uft.find(i)].append(i)
        group_size[uft.find(i)] = uft.size[uft.find(i)]
        group_map[i] = uft.find(i)

    large_groups = set([k for k, v in group_size.items() if v > min(max_N, max_dist+1)])

    tmp_df["group"] = tmp_df["query_ix"].map(group_map)
    tmp_df["size"] = tmp_df["group"].map(group_size)
    tmp_df["left"] = np.minimum(tmp_df["query_ix"], tmp_df["candidate_ix"])
    tmp_df["right"] = np.maximum(tmp_df["query_ix"], tmp_df["candidate_ix"])
    tmp_df = tmp_df.sort_values(by="pred", ascending=False).drop_duplicates(["left", "right"])

    graphs = {k: nx.Graph() for k in large_groups}
    for large_group in large_groups:
        for member in group_members[large_group]:
            graphs[large_group].add_node(member)

    neighbors = defaultdict(list)
    for l, r, s in tmp_df[tmp_df["group"].isin(large_groups)][["left", "right", "pred"]].values:
        l, r = int(l), int(r)
        g = uft.find(l)
        neighbors[l].append((r, s))
        neighbors[r].append((l, s))
        graphs[g].add_edge(l, r)

    shortest_paths = {g: {k: d for k, d in nx.all_pairs_shortest_path_length(graphs[g])} for g in large_groups if len(group_members[g]) < 100}

    scores = []
    for i in eval_points:
        true_count = test_df.at[i, "true_count"]
        qp = pids[i]
        g = uft.find(i)
        preds = []
        if g in large_groups:
            if g in shortest_paths.keys():
                preds = []
                for n, d in shortest_paths[g][i].items():
                    if d <= max_dist:
                        preds.append(n)
            if (len(preds) > max_N) or (not g in shortest_paths.keys()):
                searched = set()
                heapq = [(-1., 0, i)]
                while len(heapq) and (len(searched) < max_N):
                    _, step, x = heappop(heapq)
                    if x in searched:
                        continue
                    searched.add(x)
                    if step >= max_dist:
                        continue
                    for n, s in neighbors[x]:
                        if n in searched:
                            continue
                        heappush(heapq, (-s, step+1, n))
                preds = list(searched)
        else:
            preds = group_members[g]
        correct = sum([qp == pids[j] for j in preds])
        scores.append(correct / (true_count + len(preds) - correct))

    score = np.mean(scores)
    if score > best_score:
        best_score = score
        best_th = th
print(best_th, best_score)

In [ ]:
#lgb_model.save_model(nadare_feature_dir + "lgb_model_mod0.lgb")
import seaborn as sns

plt.figure(figsize=(20, 30))
sns.barplot(x=0, y="index", data=f_df.reset_index())
#plt.savefig(nadare_feature_dir + "feature_importances_mod0.png")
plt.show()


In [ ]:
import lightgbm as lgb

lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 2**11 - 1,
    "learning_rate" : 0.1,
    "bagging_freq": 1,
     "bin_construct_sample_cnt": 200000,
    "lambda_l1": 1.,
    "lambda_l2": 1.,
    "bagging_fraction" : 0.9,
    "feature_fraction" : 0.6,
    "seed": 0}

dev_data = lgb.Dataset(dev_data_df[feature_cols], label=dev_data_df["label"].astype(np.float32), weight=dev_data_df["weight"].values)
val_data = lgb.Dataset(val_data_df[feature_cols], label=val_data_df["label"].astype(np.float32))

lgb_model = lgb.train(lgb_params,
                      dev_data,
                      100000,
                      valid_sets = [dev_data, val_data],
                      early_stopping_rounds=10,
                      verbose_eval = 10
                      )
f_df = pd.DataFrame(lgb_model.feature_importance(importance_type="gain"), index=feature_cols).sort_values(by=0, ascending=False)


val_data_df["pred"] = lgb_model.predict(val_data_df[feature_cols])

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import warnings
from heapq import heappop, heappush
from collections import defaultdict
from tqdm.notebook import tqdm
warnings.simplefilter('ignore')

max_dist = 3
max_N = 64
eval_points = val_data_df["query_ix"].unique()
pids = test_df["pid"].values

best_score = 0
best_th = 0

pred_df = val_data_df[["query_ix", "candidate_ix", "pred", "rank"]]#.query("rank <= 16")

for th in tqdm(np.linspace(0.5, 0.9, 40)):
    tmp_df = pred_df[pred_df["pred"] >= th][["query_ix", "candidate_ix", "pred"]]
    uft = UnionFind(len(test_df))

    for ix, nix in tmp_df[["query_ix", "candidate_ix"]].values:
        uft.unite(ix, nix)
    group_members = defaultdict(list)
    group_size = defaultdict(int)
    group_map = {}
    for i in range(len(test_df)):
        group_members[uft.find(i)].append(i)
        group_size[uft.find(i)] = uft.size[uft.find(i)]
        group_map[i] = uft.find(i)

    large_groups = set([k for k, v in group_size.items() if v > min(max_N, max_dist+1)])

    tmp_df["group"] = tmp_df["query_ix"].map(group_map)
    tmp_df["size"] = tmp_df["group"].map(group_size)
    tmp_df["left"] = np.minimum(tmp_df["query_ix"], tmp_df["candidate_ix"])
    tmp_df["right"] = np.maximum(tmp_df["query_ix"], tmp_df["candidate_ix"])
    tmp_df = tmp_df.sort_values(by="pred", ascending=False).drop_duplicates(["left", "right"])

    graphs = {k: nx.Graph() for k in large_groups}
    for large_group in large_groups:
        for member in group_members[large_group]:
            graphs[large_group].add_node(member)

    neighbors = defaultdict(list)
    for l, r, s in tmp_df[tmp_df["group"].isin(large_groups)][["left", "right", "pred"]].values:
        l, r = int(l), int(r)
        g = uft.find(l)
        neighbors[l].append((r, s))
        neighbors[r].append((l, s))
        graphs[g].add_edge(l, r)

    shortest_paths = {g: {k: d for k, d in nx.all_pairs_shortest_path_length(graphs[g])} for g in large_groups if len(group_members[g]) < 100}

    scores = []
    for i in eval_points:
        true_count = test_df.at[i, "true_count"]
        qp = pids[i]
        g = uft.find(i)
        preds = []
        if g in large_groups:
            if g in shortest_paths.keys():
                preds = []
                for n, d in shortest_paths[g][i].items():
                    if d <= max_dist:
                        preds.append(n)
            if (len(preds) > max_N) or (not g in shortest_paths.keys()):
                searched = set()
                heapq = [(-1., 0, i)]
                while len(heapq) and (len(searched) < max_N):
                    _, step, x = heappop(heapq)
                    if x in searched:
                        continue
                    searched.add(x)
                    if step >= max_dist:
                        continue
                    for n, s in neighbors[x]:
                        if n in searched:
                            continue
                        heappush(heapq, (-s, step+1, n))
                preds = list(searched)
        else:
            preds = group_members[g]
        correct = sum([qp == pids[j] for j in preds])
        scores.append(correct / (true_count + len(preds) - correct))

    score = np.mean(scores)
    if score > best_score:
        best_score = score
        best_th = th
print(best_th, best_score)

In [ ]:
lgb_model.save_model(nadare_feature_dir + "lgb_model_mod0.lgb")
import seaborn as sns

plt.figure(figsize=(20, 30))
sns.barplot(x=0, y="index", data=f_df.reset_index())
plt.savefig(nadare_feature_dir + "feature_importances_mod0.png")
plt.show()


In [ ]:
@tf.function(experimental_relax_shapes=True)
def calc_rouge_1_2(q_ix, c_ix):
    q_code = tf.gather(name_bow_rag_col, q_ix).to_tensor(default_value=-1)
    c_code = tf.gather(name_bow_rag_col, c_ix).to_tensor(default_value=-2)
    q_cnt = tf.cast(tf.gather(name_bow_rag_col.row_lengths(), q_ix), "float32")
    c_cnt = tf.cast(tf.gather(name_bow_rag_col.row_lengths(), c_ix), "float32")

    is_na = tf.minimum(q_cnt, c_cnt) == 0.
    match = tf.expand_dims(q_code, axis=2) == tf.expand_dims(c_code, axis=1)
    precision = tf.math.count_nonzero(tf.math.reduce_any(match, axis=2), axis=1, dtype="float32") / (q_cnt + tf.keras.backend.epsilon())
    recall = tf.math.count_nonzero(tf.math.reduce_any(match, axis=1), axis=1, dtype="float32") / (c_cnt + tf.keras.backend.epsilon())
    harmony = 2. / ((1 / (precision + tf.keras.backend.epsilon())) + (1 / (recall + tf.keras.backend.epsilon())))
    res_1 = tf.where(tf.expand_dims(is_na, axis=-1), np.nan, tf.stack([harmony, precision, recall], axis=1))
    
    q_code = tf.gather(bi_name_bow_rag_col, q_ix).to_tensor(default_value=-1)
    c_code = tf.gather(bi_name_bow_rag_col, c_ix).to_tensor(default_value=-2)
    q_cnt = tf.cast(tf.gather(bi_name_bow_rag_col.row_lengths(), q_ix), "float32")
    c_cnt = tf.cast(tf.gather(bi_name_bow_rag_col.row_lengths(), c_ix), "float32")

    is_na = tf.minimum(q_cnt, c_cnt) == 0.
    match = tf.expand_dims(q_code, axis=2) == tf.expand_dims(c_code, axis=1)
    precision = tf.math.count_nonzero(tf.math.reduce_any(match, axis=2), axis=1, dtype="float32") / (q_cnt + tf.keras.backend.epsilon())
    recall = tf.math.count_nonzero(tf.math.reduce_any(match, axis=1), axis=1, dtype="float32") / (c_cnt + tf.keras.backend.epsilon())
    harmony = 2. / ((1 / (precision + tf.keras.backend.epsilon())) + (1 / (recall + tf.keras.backend.epsilon())))
    res_2 = tf.where(tf.expand_dims(is_na, axis=-1), np.nan, tf.stack([harmony, precision, recall], axis=1))
    
    return tf.concat([res_1, res_2], axis=-1)

In [ ]:
%%timeit
with tf.device("CPU: 0"):
    calc_rouge_1_2(q_ix, c_ix)

In [ ]:
calc_rouge_1_2(q_ix, c_ix)

In [ ]:
harmony

In [ ]:
tf.minimum(precision, recall)